# Analysis of snRNA-Seq (10x Genomics Platform)

## Bioinformatics Core Facility, University of Manchester

1. <a href=#section1>Preliminaries</a>
2. <a href=#section2>QC analysis of cells and genes</a>
3. <a href=#section3>Quality filtering of cells</a>
4. <a href=#section4>Classification of cell cycle phase</a>
5. <a href=#section5>Marking low-abundance genes</a>
6. <a href=#section6>Normalisation of read counts</a>
7. <a href=#section7>Identifying genes for feature selection (HVG)</a>
8. <a href=#section8>Data integration (donor effect correction)</a>
9. <a href=#section9>Dimensionality reduction using HVG</a>
10. <a href=#section10>Clustering of cells</a>
11. <a href=#section11>Marker gene detection</a>
12. <a href=#section12>Cell type annotation</a>

# Project summary

<div class="alert alert-info">
  <strong>About this dataset: </strong> (edits required) <br /><br />
  This notebook documents the analysis steps for single nuclei RNA sequencing dataset.
  <ul>
    <li><strong>Run:</strong> 
        <ul>
            <li>add Run Name 1 (Run ID 1)</li>
            <li>add Run Name 2 (Run ID 2)</li>
            <li>add Run Name 3 (Run ID 3)</li>
            <li>add Run Name 4 (Run ID 4)</li>
        </ul>
      <li><strong>Cell ranger binary/options:</strong> add Cell Ranger info</li>
    <li><strong>Researcher:</strong> add name</li>
    <li><strong>PI:</strong> add name</li>
    <li><strong>Analyst:</strong> add name</li>
    <li><strong>Sample name:</strong> add name</li>
  </ul>
</div>

# Project description

<div class="alert alert-info">
  edits required.
</div>

# 1. Preliminaries<a name='section1' />

- Set sample name and Cell Ranger HDF5 file path
- Load R libraries
- Set up colour palettes
- Load Cell Ranger outputs to create `SingleCellExperiment` (`sce`) objects
- Add log2 counts per million to `sce` object
- Estimate ambient contamination

## Define sample name and HDF5 path

In [ ]:
# Set sample name
sample_name <- c("W13","W08_1","W15","W08_2")

# Path to the cellranger aggr filtered HDF5
# Can be relative or full path
filtered_h5 = "filtered_feature_bc_matrix.h5"

# Path to the cellranger raw HDF5 for individual sample
# Can be relative or full path
raw_h5 <- c("W13/raw_feature_bc_matrix.h5", 
            "W08_1/raw_feature_bc_matrix.h5", 
            "W15/raw_feature_bc_matrix.h5", 
            "W08_2/raw_feature_bc_matrix.h5")

# Show info
data.frame(ID = c("aggr", sample_name), Type = c("filtered", rep("raw", length(sample_name))), 
           HDF5 = c(filtered_h5, raw_h5))

## Load libraries

The required R packages are listed below. The packages listed under R-4.0.2 are that available through the `singularity/R4.0.2` image. Additional packaged required but not available through the image are listed separately. 

The packages `bluster` and `celldex` are only available for R-4.0.3. Therefore you can choose to install them from source packages. 

- The `plotSilhouette` function used in this workflow will detect if `bluster` is installed and return a warning message if it is not, and will not generate the silhouette plot. 
- The `celldex` package offers cell type reference datasets that is previously included in `SingleR`. Under R-4.0.2, the `SingleR` will offer these datasets, whereas in R-4.0.3, one must use `celldex` instead. 

<div class="alert alert-warning">
    <strong>Warning!</strong> The <code>batchelor</code> package is required here so that we can use mutual nearest neighbors (MNN) method (<code>fastMNN</code> function) to perform batch correction for multi-sample dataset.
</div>

In [ ]:
# R-4.0.2
library(cluster)         # silhouette
library(ComplexHeatmap)
library(cowplot)         # plotColData, plotRowData, plot_grid
library(DropletUtils)    # read10xCounts
library(dplyr)
library(dynamicTreeCut)  # cutreeDynamic
library(enrichR)
library(ensembldb)
library(factoextra)      # fviz_silhouette
library(GGally)          # ggpairs
library(ggbeeswarm)      # geom_quasirandom
library(ggforce)         # gather_set_data
library(ggplot2)
library(ggrepel)         # geom_text_repel
library(HDF5Array)       # saveHDF5SummarizedExperiment
library(igraph)          # cluster_walktrap
library(KernSmooth)      # smoothScatter
library(Rtsne)           # runTSNE
library(scales)
library(scater)
library(scran)
library(uwot)            # runUMAP
library(viridis)         # scale_color_viridis

# Additional required packages but not on CSF3, available for R-4.0.2
library(AnnotationHub)      # also installs and load ExperimentHub, interactiveDisplayBase
library(batchelor)          # fastMNN, also installs ResidualMatrix
library(DelayedMatrixStats) # rowMeans2 and rowVars
library(PCAtools)           # findElbowPoint
library(pheatmap)
library(SingleR)

# Additional required packages but not on CSF3, available for R-4.0.3
# Need to install from source: install.packages("package.tar.gz", repos = NULL, type="source")
# Not loading them now, will call them when using the required functions later
#library(bluster)         # approxSilhouette
#library(celldex)         # Pokédex for Cell Types (datasets were part of SingleR in old version)

# Load tidyverse last to generate tidyverse conflicts report
library(tidyverse)

### Set default options

In [ ]:
# Set output window width
default.ww <- 80 # getOption("width")
options(width = 110)

# Set output image size
default.w <- 10   # repr.plot.width
default.h <- 7    # repr.plot.height
default.r <- 120  # repr.plot.res

# Function to reset figure size with default settings
reset.fig <- function() { 
    options(repr.plot.width = default.w, 
            repr.plot.height = default.h, 
            repr.plot.res = default.r) 
}

reset.fig()

# Function to set figure size
fig <- function(width = default.w, height = default.h, res = default.r){
     options(repr.plot.width = width, 
             repr.plot.height = height, 
             repr.plot.res = res)
}

In [ ]:
# Perform the calculation across multiple cores
bpp <- BiocParallel::MulticoreParam(workers = nthreads)

## Set up colour palette

[This](https://mokole.com/palette.html) palette generator was used to create a 30-colour palette.

In [ ]:
#cbPalette <- paletteer_d(package = "ggthemes", palette="calc", n=12)
c30 <- c(
        "#7f0000", # maroon2
        "#006400", # darkgreen
        "#808000", # olive
        "#483d8b", # darkslateblue
        "#008b8b", # darkcyan
        "#cd853f", # peru
        "#4682b4", # steelblue
        "#00008b", # darkblue
        "#8fbc8f", # darkseagreen
        "#800080", # purple
        "#b03060", # maroon3
        "#ff0000", # red
        "#ff8c00", # darkorange
        "#cdcd00", # yellow3
        "#00ff00", # lime
        "#8a2be2", # blueviolet
        "#00ff7f", # springgreen
        "#dc143c", # crimson
        "#00ffff", # aqua
        "#0000ff", # blue
        "#f08080", # lightcoral
        "#adff2f", # greenyellow
        "#da70d6", # orchid
        "#ff00ff", # fuchsia
        "#1e90ff", # dodgerblue
        "#f0e68c", # khaki
        "#90ee90", # lightgreen
        "#ff1493", # deeppink
        "#696969", # dimgray
        "#dcdcdc"  # gainsboro
)
 
pie(rep(1,30), col = c30, radius = 1)

In [ ]:
# Choosing colours for samples
c_sample_col <- c30[c(1,4,5,6)]
c_sample_col <- setNames(c_sample_col, sample_name)
c_sample_col

# Choosing colour for clusters
c_clust_col <- c30
c_clust_col <- setNames(c_clust_col, as.character(1:30))
c_clust_col

# Choosing colour for cell cycle phases
c_phase_col <- setNames(c30[c(18,15,20)], c("G1", "S", "G2M"))
c_phase_col

## Importing Cell Ranger data

We will use `read10xCounts` from `DropletUtils` package to read the UMI counts produced by Cell Ranger and automatically generate a `sce` object. The **HDF5** format enables out-of-core computation.

<div class="alert alert-info">
  <strong>Info!</strong> Requires filtered HDF5 file (filtered_feature_bc_matrix.h5).
</div>

In [ ]:
if(packageVersion("DropletUtils") == "1.8.0") {
    cdSc <- read10xCounts(filtered_h5, col.names = TRUE) # column named with the cell barcodes
} else {
    cdSc <- read10xCounts(filtered_h5, col.names = TRUE, # column named with the cell barcodes
                          BPPARAM = bpp) # parallelized for multiple samples
}
cdSc

In [ ]:
# Use the Barcode suffix to determine Sample ID
cdSc$Sample <- gsub("^[A-Z]*-", "", cdSc$Barcode)

# Change sample from character to factor type
cdSc$Sample <- factor(cdSc$Sample)
levels(cdSc$Sample) <- sample_name

# Use uniquifyFeatureNames to make feature names unique
rownames(cdSc) <- uniquifyFeatureNames(rowData(cdSc)$ID, rowData(cdSc)$Symbol)

Print number of cells in each sample.

In [ ]:
print("Number of cells in each sample:")
table(cdSc$Sample)

## Add SEQNAME to gene information

Here, we make use of `AnnotationHub` to add SEQNAME (chromosome) annotation from Ensembl DB.

In [ ]:
# Create an AnnotationHub object
ah = AnnotationHub()

# To display Human EnsDb if required
#hs <- query(ah, c("EnsDb", "Ensembl", "Homo sapiens"))
#hs

# To display Mouse EnsDb if required
#mm <- query(ah, c("EnsDb", "Ensembl", "Mus musculus"))
#mm

In [ ]:
# Select the EnsDb corresponds to the correct Reference Package used by Cell Ranger 
#  3.0.0: AH64446 | Ensembl 93 EnsDb for Homo Sapiens
#  3.0.0: AH64461 | Ensembl 93 EnsDb for Mus musculus
# 2020-A: AH75011 | Ensembl 98 EnsDb for Homo sapiens
# 2020-A: AH75036 | Ensembl 98 EnsDb for Mus musculus

ens <- AnnotationHub()[["AH64446"]]

In [ ]:
# Add SEQNAME
rowData(cdSc)$SEQNAME <- mapIds(ens, keys = rowData(cdSc)$ID, keytype = "GENEID", column = "SEQNAME")

# Number of genes in each chromosome
table(rowData(cdSc)$SEQNAME)

Access the count matrix, which is a `DelayedMatrix` class for HDF5 dataset.

In [ ]:
# The rows are genes and columns are cells.
counts(cdSc)[1:5,1:5] # same as assay(cdSc, "counts")

## Add log2 counts per million

This normalises based on all cells in the `sce` object prior to any QC filtering. Normalisation will be performed again on the filtered `sce` object later.

In [ ]:
# This normalises prior to any QC filtering. So gets redone, but sometimes requested.
logcounts(cdSc) <- log2(calculateCPM(cdSc) + 1)

# A new "logcounts" assay is added
cdSc

__Normalisation notes__ Syed note about an older version of scater which automatically normalised the counts in its "log10counts". Not doing that at the moment.
Here we use log2-counts-per-million with an offset of 1 as the expression values. We have to note that the CPM for scater is different than the regular CPM calculation as we are considering the step-size here. 

The value of the log-CPMs is explained by adding a prior count to avoid undefined values after the log-transformation, multiplying by a million, and dividing by the mean library size. This size factors are used to define the effective library sizes. This is done by scaling all size factors such that the mean scaled size factor is equal to the mean sum of counts across all features. The effective library sizes are then used to in the denominator of the CPM calculation. The way that `scater` calculates the log-normalized counts are

```
lib.sizes <- colSums(counts(example_sceset))
lib.sizes <- lib.sizes/mean(lib.sizes)
log2(counts(example_sceset)[1,]/lib.sizes+1)
```

## Estimate ambient contamination

Ambient contamination is a phenomenon that is generally most pronounced in massively multiplexed scRNA-seq protocols. Briefly, extracellular RNA (most commonly released upon cell lysis) is captured along with each cell in its reaction chamber, contributing counts to genes that are not otherwise expressed in that cell.

If our snRNA-seq libraries are of high quality, we can assume that any mitochondrial “expression” is due to contamination from the ambient solution. Below we use the `controlAmbience` function from `DropletUtils` package to estimate the proportion of ambient contamination for each gene.

<div class="alert alert-warning">
    <strong>Warning!</strong> The <code>DropletUtils</code> used here is v1.10.1. From v1.11.7 onwards, the <code>emptyDrops</code> and <code>estimateAmbience</code> functions  is updated/changed which will improve the speed a lot. The <code>estimateAmbience</code> function will be soft-deprecated, and replaced by <code>ambientProfileEmpty</code>. <strong>This section of the code will need to be changed in the future.</strong>
</div>

### Create pseudo-bulk expression profiles

We create the pseudo-bulk expression profiles by summing counts together for all cells from the same sample.

<div class="alert alert-info">
  <strong>Info!</strong> The pseudo-bulk expression calculation uses the filtered sce object (<code>cdSc</code>).
</div>

In [ ]:
summed.cdSc <- aggregateAcrossCells(cdSc, ids = cdSc$Sample, BPPARAM = bpp)
summed.cdSc

### Load HDF5 and estimate the ambient profile

<div class="alert alert-warning">
    <strong>Warning!</strong> Using <code>DelayedMatrix</code> object as input is extremely slow when running <code>estimateAmbience</code>. Here we convert count matrix from <code>DelayedMatrix</code> to <code>dgCMatrix</code> class first before analysis.
</div>

In [ ]:
ambient <- vector("list", ncol(summed.cdSc))

for (s in seq_along(ambient)) {
    print(paste("Sample:", sample_name[s]))
    
    # Load HDF5
    # Adding colnames to sce (col.names = TRUE) is very important in speeding up the as() operation below
    sce <- read10xCounts(raw_h5[s], sample.names = sample_name[s], col.names = TRUE)
    print(sce)
    
    sce$Sample <- factor(sce$Sample)
    rownames(sce) <- uniquifyFeatureNames(rowData(sce)$ID, rowData(sce)$Symbol)
    
    # Convert DelayedMatrix to dgCMatrix for massive speed gain
    mat <- as(counts(sce, withDimnames = FALSE), "dgCMatrix")
    rownames(mat) <- rownames(sce)

    # Estimate the transcript proportions in the ambient solution
    ambient[[s]] <- estimateAmbience(mat, round = FALSE, good.turing = FALSE)
    print(summary(ambient[[s]]))
    flush.console() # Ensures display of output is current
}

# Cleaning up the output for pretty printing
ambient <- do.call(cbind, ambient)
colnames(ambient) <- seq_len(ncol(ambient))
head(ambient)

### Estimate the ambient contribution from controls

Control features should be those that cannot be expressed and thus fully attributable to ambient contamination. For single-nuclei sequencing, mitochondrial transcripts are used as control features under the assumption that all high-quality libraries are stripped nuclei.

In [ ]:
# Add missing functions if DropletUtils is too old
if(packageVersion("DropletUtils") == "1.8.0") {
    controlAmbience <- function(y, ambient, features, mode=c("scale", "profile", "proportion")) {
        if (is.null(dim(y))) {
            y <- cbind(y)
        }
        if (is.null(dim(ambient))) {
            ambient <- matrix(ambient, nrow(y), ncol(y), dimnames = list(names(ambient), NULL))
        }
        if (!identical(rownames(y), rownames(ambient))) {
            warning("'y' and 'ambient' do not have the same row names")
        }

        if (!is.list(features)) {
            features <- list(features)
        } 
        sum.y <- sumCountsAcrossFeatures(y, features)
        sum.a <- sumCountsAcrossFeatures(ambient, features)
        props <- sum.y/sum.a
        scaling <- apply(props, 2, min)

        mode <- match.arg(mode)
        if (mode == "scale") {
            scaling
        } else {
            scaled.ambient <- t(t(ambient) * scaling)
            if (mode == "profile") {
                scaled.ambient
            } else {
                .clean_amb_props(scaled.ambient, y)
            }
        }
    }
    
    .clean_amb_props <- function(s, y) {
        p <- s/y
        p[p > 1] <- 1
        p[y == 0] <- NaN # for fairness's sake.
        p
    }
}

In [ ]:
nuclei <- counts(summed.cdSc) # pseudo-bulk expression
is.mito <- rowData(cdSc)$SEQNAME == "MT"

# Estimate the proportion of ambient contamination for each gene
contam <- controlAmbience(nuclei, ambient, features = is.mito,  mode = "proportion")
head(contam)

summary(rowMeans(as.matrix(contam), na.rm = TRUE))

We create a plot to show the percentage of counts in the nuclei that are attributed to contamination from the ambient solution. Each point represents a gene and mitochondrial genes are highlighted in red.

In [ ]:
# Percentage of counts in the nuclei of the dataset that are attributed to contamination from the ambient solution.
# Each point represents a gene and mitochondrial genes are highlighted in red.
plot(log10(nuclei+1), contam*100, col = ifelse(is.mito, "red", "grey"), pch = 16, 
     xlab = "Log-nuclei expression", ylab = "Contamination (%)")

In [ ]:
# Show mito gene estimates
as.data.frame(contam[is.mito,]) %>% 
    add_column(rowMeans = rowMeans(as.data.frame(contam[is.mito,]), na.rm = TRUE)) %>%
    arrange(rowMeans)

# Set lowest mean mito gene estimate as cutoff
contam.cutoff <- as.data.frame(contam[is.mito,]) %>% 
    add_column(rowMeans = rowMeans(as.data.frame(contam[is.mito,]), na.rm = TRUE)) %>%
    arrange(rowMeans) %>% pull(rowMeans) %>% head(1)
print(paste("Ambient contamination cutoff:", round(contam.cutoff, 4)))

# Keep genes in which less than N% of the counts are ambient-derived
non.ambient <- rowMeans(as.data.frame(contam), na.rm = TRUE) < contam.cutoff
summary(non.ambient)

okay.genes <- names(non.ambient)[which(non.ambient)]
print(paste("Number of genes passed the ambient contamination cutoff:", length(okay.genes)))


# 2. QC analysis of cells and genes<a name='section2' />

__View information related with cells using:__ `colData(cdSc)`

__View information related with genes using:__ `rowData(cdSc)`

### Define mitochondrial genes 

In [ ]:
## Works on both Mouse & Human
#is.mito <- grepl("^MT-|^mt-", rowData(cdSc)$Symbol)

is.mito <- rowData(cdSc)$SEQNAME == "MT"
rowData(cdSc)$is_mito <- is.mito

print(paste("Number of annotated mitochondrial genes =", sum(is.mito)))

In [ ]:
rowData(cdSc)$Symbol[is.mito]

### Add QC metrics

In [ ]:
# Add Cell QC
if(packageVersion("scater") == "1.16.2") {
    cdSc <- addPerCellQC(cdSc, subsets = list(Mt = is.mito), BPPARAM = bpp)
} else {
    cdSc <- addPerCellQC(cdSc, percent.top = c(50, 100, 200, 500), subsets = list(Mt = is.mito), BPPARAM = bpp)
}
colData(cdSc)$log10GenesPerUMI <- log10(colData(cdSc)$detected+1) / log10(colData(cdSc)$sum+1)

# Mark potentially problematic genes due to ambient contamination
rowData(cdSc)$is_ambient <- !rownames(cdSc) %in% okay.genes

# Add feature QC
cdSc <- addPerFeatureQC(cdSc, detection_limit = 0, BPPARAM = bpp)
rowData(cdSc)$n_cells_by_counts <- rowData(cdSc)$detected/100 * ncol(cdSc)
rowData(cdSc)$pct_dropout <- 100 - rowData(cdSc)$detected

#### View information related with cells

We will use `colData()` function to access the metadata related with cells.

**Notes on `colData()` elements**

- sum - total number of UMIs
- detected - number of genes detected above detection limit (default 0)
- percent.top_XX - percentage of each cell's count sum assigned to each subset
- subsets_Mt_sum - number of UMIs mapped to mitochondrial genes
- subsets_Mt_detected - number of mitochondrial genes detected (for human: range from 0 ~ 13 genes)
- subsets_Mt_percent - percentage of each cell's count sum assigned to mitochondrial genes
- log10GenesPerUMI - genes detected per UMI

In [ ]:
#Checks: should  13 columns (used to be 12, added log10GenesPerUMI)
print(paste("Number of columns in colData =", length(names(colData(cdSc)))))

head(colData(cdSc), 5)

#### View information related with genes

We use `rowData()` function to access the metadata related with genes.

**Notes on `rowData()` elements**

- is_mito - mitochondrial genes
- is_ambient - genes affected by ambient contamination
- mean - mean counts for each genes across all cells
- detected - percentage of cells a gene is detected above detection limit (default 0)
- n_cells_by_counts - number of cells a gene detected
- pct_dropout - percentage of dropouts, i.e. `1 - detected` 

*Dropouts are statistical negatives, could be true negatives or false negatives (not detected properly)*

In [ ]:
#Checks: should 10 (used to be 7, nowadded SEQNAME, is_mito and is_ambient)
print(paste("Number of columns in rowData =", length(names(rowData(cdSc)))))

head(rowData(cdSc), 5)

## Plots on cell QC metrics

Low-quality cells need to be identified and removed to ensure that the technical effects do not distort downstream analysis results. Three common measures of cell quality are:

- the total number of UMIs or __library size__ per cell
- the __number of detectable features (i.e. genes)__ in each cell library
- proportion of __mitochondrial genes__ in each cell library

The __library size__ is defined as the total sum of UMIs across all genes. Cells with relatively small library sizes are considered to be of low quality as the RNA has not been efficiently captured (i.e., converted into cDNA and amplified) during library preparation. 

The __number of detectable genes__ in each cell is defined as the number of genes above detection limit (default is with non-zero counts) for that cell. Any cell with very few expressed genes is likely to be of poor quality as the diverse transcript population has not been successfully captured. 

High proportions of reads mapping to __mitochondrial genes__ are indicative of poor-quality cells ([Ilicic et al., 2016](https://f1000research.com/articles/5-2122/v2#ref-14); [Islam et al., 2014](https://f1000research.com/articles/5-2122/v2#ref-16)), possibly because of increased apoptosis and/or loss of cytoplasmic RNA from lysed cells. 

Below are different plots to visualise the summary statistics.

In [ ]:
# Print library size summary
summary(colData(cdSc)$sum)

# Plot histogram
ggplot(as.data.frame(colData(cdSc)), aes(x = sum, fill = Sample)) + 
    geom_histogram(color = "white", alpha = 0.6, size = 0.3, bins = 50) +
    scale_x_continuous(labels = unit_format(unit = "K", scale = 1e-3)) +
    scale_fill_manual(values = c_sample_col) + 
    guides(fill = guide_legend(override.aes = list(alpha = 1))) +
    theme_classic(base_size = 20) +
    labs(x = "Library size", y = "Number of cells", title = "Histogram of library size")

In [ ]:
# Print detected genes summary
summary(colData(cdSc)$detected)

# Plot histogram
ggplot(as.data.frame(colData(cdSc)), aes(x = detected, fill = Sample)) +
    geom_histogram(color = "white", alpha = 0.6, size = 0.3, bins = 50) +
    scale_x_continuous(labels = unit_format(unit = "K", scale = 1e-3)) +
    scale_fill_manual(values = c_sample_col) + 
    guides(fill = guide_legend(override.aes = list(alpha = 1))) +
    theme_classic(base_size = 20) +
    labs(x = "Number of detected genes", y = "Number of cells", title = "Histogram of detected genes")

In [ ]:
# Print mitochondrial proportion summary
summary(colData(cdSc)$subsets_Mt_percent)

print("Number of cells with 0% mitochondrial content:")
data.frame(Sample = colData(cdSc)$Sample, ZeroMito = colData(cdSc)$subsets_Mt_percent == 0) %>%
    count(Sample, ZeroMito, name = "Count") %>% group_by(Sample) %>% 
    mutate(Percentage = round(prop.table(Count)*100, 2))

# Plot histogram
ggplot(as.data.frame(colData(cdSc)), aes(x = subsets_Mt_percent, fill = Sample)) +
    geom_histogram(color = "white", alpha = 0.6, size = 0.3, bins = 50) +
    scale_fill_manual(values = c_sample_col) + 
    guides(fill = guide_legend(override.aes = list(alpha = 1))) +
    theme_classic(base_size = 20) +
    labs(x = "Mitochondrial proportion (%)", y = "Number of cells", 
         title = "Histogram of Mitochondrial proportion")

In snRNA-seq data, the loss of the cytoplasm means that the stripped nuclei should not contain any mitochondrial transcripts. High-quality nuclei __should not contain any mitochondrial transcripts__; the presence of any mitochondrial counts in a library indicates that the removal of the cytoplasm was not complete, possibly introducing irrelevant heterogeneity in downstream analyses.

__For scRNA-seq:__

- A mean below 10% is very good.
- A mean above 25% is no good.

<div class="alert alert-info">
  <strong>About this dataset: </strong> (edits required)
  <ul>
    <li>Most of the cells in this dataset have incompletely stripped nuclei.</li>
    <li>We will perform filtering to remove cells with high mitochondrial proportion.</li>
  </ul>
</div>

### Novelty

Visualise the overall novelty of the gene expression by showing log10 genes detected per UMI against number of cells. Generally, we expect the novelty score to be above 0.80.

In [ ]:
# Print Novelty summary
summary(colData(cdSc)$log10GenesPerUMI)

# Plot histogram
ggplot(as.data.frame(colData(cdSc)), aes(x = log10GenesPerUMI, fill = Sample)) +
    geom_histogram(color = "white", alpha = 0.6, size = 0.3, bins = 50) +
    scale_fill_manual(values = c_sample_col) + 
    guides(fill = guide_legend(override.aes = list(alpha = 1))) +
    theme_classic(base_size = 20) +
    labs(x = "log10 nGene per log10 nUMI", y = "Number of cells", 
         title = "Histogram of complexity of RNA species")

<div class="alert alert-warning">
    <strong>Warning!</strong> There were issues with the colour mapping args with the old <code>scater</code> installed on CSF3, a function <code>oldScaterPlotFix</code> was created to fix plots created with <code>plotColData</code>, <code>plotRowData</code> and <code>plotReducedDim</code>.
</div>

In [ ]:
# Fix old scater's plot_central() issues with the colour mapping arg
oldScaterPlotFix <- function(p) {
    if(packageVersion("scater") == "1.16.2") {
        # edit layer
        d <- data.frame(i = integer(), j = integer(), aesthetics = character())
        for(i in seq_len(length(p$layers))) {
            for(j in seq_len(length(names(p$layers[[i]]$mapping)))) {
                d <- rbind(d, data.frame(i = i, j = j, aes = names(p$layers[[i]]$mapping[j])))
            }
        }
        # If colour is not present, change fill to colour
        if(nrow(d[grep("colour", d$aes),]) == 0) {
            i <- d[grep("fill", d$aes),]$i
            j <- d[grep("fill", d$aes),]$j
            names(p$layers[[i]]$mapping)[j] <- "colour"
            p$layers[[i]]$aes_params[['colour']] <- NULL
            p$layers[[i]]$aes_params[['fill']] <- "grey20"
            p$layers[[i]]$aes_params[['shape']] <- p$layers[[i]]$geom$default_aes[['shape']]
        }
    
        # edit scales
        d <- data.frame(i = integer(), j = integer(), aesthetics = character())
        for(i in seq_len(length(p$scales$scales))) {
            for(j in seq_len(length(p$scales$scales[[i]]$aesthetics))) {
                d <- rbind(d, data.frame(i = i, j = j, aes = p$scales$scales[[i]]$aesthetics[j]))
            }
        }
        # If colour is not present, change fill to colour
        if(nrow(d[grep("colour", d$aes),]) == 0) {
            i <- d[grep("fill", d$aes),]$i
            j <- d[grep("fill", d$aes),]$j
            p$scales$scales[[i]]$aesthetics[j] <- "colour"
        }
    }
    p
}

In [ ]:
p <- plotColData(cdSc, x = "sum", y = "detected", colour_by = "Sample", other_fields = "Sample", 
                 point_alpha = 0.3, theme_size = 20) +
    facet_wrap(~ Sample) +
    scale_x_continuous(labels = unit_format(unit = "K", scale = 1e-3)) +
    scale_color_manual(values = c_sample_col) + theme(legend.position = "none") +
    labs(x = "Library size", y = "Number of detected genes")

fig(width = 16, height = 8)
oldScaterPlotFix(p)
reset.fig()

In [ ]:
p <- plotColData(cdSc, x = "sum", y = "subsets_Mt_percent", colour_by = "Sample", other_fields = "Sample", 
                 point_alpha = 0.3, theme_size = 20) +
    facet_wrap(~ Sample) +
    scale_x_continuous(labels = unit_format(unit = "K", scale = 1e-3)) +
    scale_color_manual(values = c_sample_col) + theme(legend.position = "none") +
    labs(x = "Library size", y = "Mitochondrial proportion (%)")

fig(width = 16, height = 8)
oldScaterPlotFix(p)
reset.fig()

In [ ]:
p <- plotColData(cdSc, x = "detected", y = "subsets_Mt_percent", colour_by = "Sample", other_fields = "Sample", 
                 point_alpha = 0.3, theme_size = 20) +
    facet_wrap(~ Sample) +
    scale_x_continuous(labels = unit_format(unit = "K", scale = 1e-3)) +
    scale_color_manual(values = c_sample_col) + theme(legend.position = "none") +
    labs(x = "Number of detected genes", y = "Mitochondrial proportion (%)")

fig(width = 16, height = 8)
oldScaterPlotFix(p)
reset.fig()

Show in log10 scale.

In [ ]:
log10_breaks <- trans_breaks("log10", function(x) 10^x)
log10_labels <- trans_format("log10", math_format(10^.x))

p <- plotColData(cdSc, x = "sum", y = "detected", colour_by = "subsets_Mt_percent", 
                 other_fields = "Sample", point_alpha = 0.3, theme_size = 20) +
    facet_wrap(~ Sample) +
    scale_x_log10(breaks = log10_breaks, labels = log10_labels) +
    scale_y_log10(breaks = log10_breaks, labels = log10_labels) +
    guides(color = guide_colourbar(title = "Mitochondrial\nproportion (%)")) +
    labs(x = "Library size", y = "Number of detected genes")

fig(width = 16, height = 8)
oldScaterPlotFix(p)
reset.fig()

In [ ]:
p <- plotColData(cdSc, x = "sum", y = "detected", colour_by = "subsets_Mt_percent", size_by = "subsets_Mt_sum", 
                 other_fields = "Sample", point_alpha = 0.3, theme_size = 20) +
    facet_wrap(~ Sample) +
    scale_x_log10(breaks = log10_breaks, labels = log10_labels) +
    scale_y_log10(breaks = log10_breaks, labels = log10_labels) +
    guides(color = guide_colourbar(title = "Mitochondrial\nproportion (%)"), 
           size = guide_legend(title = "Mitochondrial\ncounts", 
                               override.aes = list(fill = "black", alpha = 1))) +
    labs(x = "Library size", y = "Number of detected genes")

fig(width = 16, height = 8)
oldScaterPlotFix(p)
reset.fig()

Cells on the linear pattern show a good correlation between total counts and the number of gene features with counts. Problematic cells deviate from the linear trend.

Cells with a very high proportion of mitchondrial reads are problematic (e.g. in apoptosis) ([Ilicic et al., 2016](https://f1000research.com/articles/5-2122/v2#ref-14); [Islam et al., 2014](https://f1000research.com/articles/5-2122/v2#ref-16)). They will be removed by the QC filtering.

In [ ]:
p <- plotColData(cdSc, x = "sum", y = "detected", colour_by = "Sample", other_fields = "Sample", 
                 point_alpha = 0.3, theme_size = 20) +
    facet_wrap(~ Sample) +
    scale_x_log10(breaks = log10_breaks, labels = log10_labels) +
    scale_y_log10(breaks = log10_breaks, labels = log10_labels) +
    scale_color_manual(values = c_sample_col) + theme(legend.position = "none") +
    labs(x = "Library size", y = "Number of detected genes")

fig(width = 16, height = 8)
oldScaterPlotFix(p)
reset.fig()

## Plots on gene QC metrics

In [ ]:
p <- plotRowData(cdSc, x = "mean", y = "detected", colour_by = "is_mito", size_by = "is_mito", theme_size = 20) + 
    scale_x_log10(breaks = log10_breaks, labels = log10_labels) +
    scale_color_manual(values = c("cyan","red")) +
    guides(color = guide_legend(title = "Mito Gene", override.aes = list(size = 4, alpha = 1)), 
           size = guide_legend(title = "Mito Gene")) +
    labs(x = "Mean counts across all cells", y = "Proportion of expressing cell (%)")

oldScaterPlotFix(p)

Mitochondrial genes (marked in red) often appear at the top right corner of the figure since they are highly expressed in most cells. It is only a problem if these genes make up the majority of a cell's reads (more than 80%). These problematic cells are removed further down this workflow.

In [ ]:
p <- plotRowData(cdSc, x = "mean", y = "detected", colour_by = "is_ambient", theme_size = 20) +
    scale_x_log10(breaks = log10_breaks, labels = log10_labels) +
    scale_color_manual(values = c("cyan","red")) +
    guides(color = guide_legend(title = "Ambient\ncontamination", override.aes = list(size = 4, alpha = 1)), 
           size = guide_legend(title = "Ambient\ncontamination")) +
    labs(x = "Mean counts across all cells", y = "Proportion of expressing cell (%)")

oldScaterPlotFix(p)

Genes affected by ambient contamination are marked in red.

In [ ]:
p <- plotRowData(cdSc, x = "mean", y = "n_cells_by_counts", colour_by = "pct_dropout", 
                 point_alpha = 0.3, theme_size = 20) +
    scale_x_log10(breaks = log10_breaks, labels = log10_labels) +
    guides(color = guide_colourbar(title = "Dropouts (%)")) +
    labs(x = "Mean counts across all cells", y = "Number of expressing cells")

oldScaterPlotFix(p)

__This plot__ shows log10 mean counts of genes (x-axis) versus number of cells expressing that gene (y-axis). Generally, in single cell datasets there are some genes with very low, or very high, total counts, which accounts for the S shape of the plot.

__Dropouts__ is the proportion of cells a gene is **not** detected. As expected, genes with low mean count have higher percentage of dropouts.

# 3. Quality filtering of cells<a name='section3' />
__Picking thresholds for filtering out poor cells__ is not straightforward for different metrics as their absolute values depend on the protocol and biological system. For example, sequencing to greater depth will lead to more reads, regardless of the quality of the cells. To obtain an adaptive threshold, the assumption made here is that most of the dataset consists of high-quality cells. Plots to facilitate picking thresholds for cell cutoffs are below.

### The distribution of the UMIs, number of detectable genes,  mitochondrial proportion and novelty.

The dotted line represents the threshold which is the **N** Median Absolute Deviation (MAD). Outlier cells are defined as those that are **N** MADs away (can be *lower*, *higher* or *both*) from the median.

For the library size, detectable genes and novelty, cells on the left of the thresholds (i.e. lower) would be filtered out. For mitochondrial proportion, cells on the right of the thresholds (i.e. higher) would be filtered out.

Finally, cells are generally removed for having mitochondrial proportion above 5 MAD. Sometime when an experiment has very low percentage of mitochondrial reads, no removal is necessary.

### Considering experimental factors

More complex studies will involve batches of cells generated with different experimental parameters (e.g., sequencing depth, different donors, etc). It makes little sense to compute medians and MADs from a mixture distribution containing samples from multiple batches. In such cases, the adaptive strategy should be applied to each batch separately. If cells from all batches have been merged into a single `SingleCellExperiment`, the `batch=` argument should be used to ensure that outliers are identified within each batch. This allows `isOutlier()` to accommodate systematic differences in the QC metrics across batches.

## Library Size

In [ ]:
# Change this setting to adjust and show different threshold
mad1 <- 2.5
libsize.drop <- isOutlier(cdSc$sum, nmads = mad1, batch = cdSc$Sample, type = "lower", log = TRUE)
cut_off_reads <- attr(libsize.drop, "thresholds")["lower",]
cut_off_reads <- cut_off_reads[levels(cdSc$Sample)] # reorder to levels
round(cut_off_reads,2)

# Set minimum library size cutoff at 500 if the MAD cutoff is lower than 500
cut_off_reads <- ifelse(cut_off_reads < 500, 500, cut_off_reads)
round(cut_off_reads,2)

libsize.drop <- c()
for(i in 1:length(cut_off_reads)) {
    libsize.drop <- c(libsize.drop, cdSc$sum[cdSc$Sample == names(cut_off_reads[i])] < cut_off_reads[i])
}

as.data.frame(colData(cdSc)) %>% 
    left_join(data.frame(thresholds = cut_off_reads, Sample = names(cut_off_reads))) %>%
    ggplot(aes(x = sum, fill = factor(Sample))) +
    geom_density(color = NA, alpha = 0.5) + facet_wrap(~ Sample) +
    geom_vline(aes(xintercept = thresholds), colour = "red", linetype = "longdash") +
    geom_text(aes(x = thresholds, label = round(thresholds, 2)), y = Inf, vjust = 2, hjust = -0.2, size = 5) +
    theme_classic(base_size = 20, base_family = "mono") + scale_fill_manual(values = c_sample_col) +
    scale_x_log10(breaks = log10_breaks, labels = log10_labels) +
    theme(legend.position = "none") +
    labs(x = "Library size", title = "Total count", fill = "Sample", 
         subtitle = paste(sum(libsize.drop), "cell(s) to the left of cutoff have too low UMI count"))

as.data.frame(colData(cdSc)) %>% cbind(., libsize.drop) %>%
    rename(LowCount = libsize.drop) %>%
    ggplot(aes(x = factor(Sample), y = sum)) + geom_violin(size = 1) +
    geom_jitter(aes(color = LowCount), alpha = 0.3, size = 0.1, 
                position = position_jitter(height = 0, width = 0.15, seed = 123)) +
    theme_classic(base_size = 20) + scale_color_manual(values = c("black", "red")) +
    scale_y_log10(breaks = log10_breaks, labels = log10_labels) +
    guides(color = guide_legend(override.aes = list(size = 4, alpha = 1))) +
    labs(x = "Sample", y = "Library size", title = "Total count", fill = "Sample",
         subtitle = paste(sum(libsize.drop), "cell(s) coloured in red have too low UMI count"))

## Number of detected genes

In [ ]:
# Change this setting to adjust and show different threshold
mad2 <- 3
feature.drop <- isOutlier(cdSc$detected, nmads = mad2, batch = cdSc$Sample, type = "lower", log = TRUE)
cut_off_genes <- attr(feature.drop, "thresholds")["lower",]
cut_off_genes <- cut_off_genes[levels(cdSc$Sample)] # reorder to levels
round(cut_off_genes,2)

# Set minimum detected genes cutoff at 250 if the MAD cutoff is lower than 250
cut_off_genes <- ifelse(cut_off_genes < 250, 250, cut_off_genes)
round(cut_off_genes,2)

feature.drop <- c()
for(i in 1:length(cut_off_genes)) {
    feature.drop <- c(feature.drop, cdSc$detected[cdSc$Sample == names(cut_off_genes[i])] < cut_off_genes[i])
}

as.data.frame(colData(cdSc)) %>% 
    left_join(data.frame(thresholds = cut_off_genes, Sample = names(cut_off_genes))) %>%
    ggplot(aes(x = detected, fill = factor(Sample))) + 
    geom_density(color = NA, alpha = 0.5) + facet_wrap(~ Sample) +
    geom_vline(aes(xintercept = thresholds), colour = "red", linetype = "longdash") +
    geom_text(aes(x = thresholds, label = round(thresholds, 2)), y = Inf, vjust = 2, hjust = -0.2, size = 5) +
    theme_classic(base_size = 20) + scale_fill_manual(values = c_sample_col) +
    scale_x_log10(breaks = log10_breaks, labels = log10_labels) +
    theme(legend.position = "none") +
    labs(x = "Number of detected genes", title = "Total detected genes", fill = "Sample", 
         subtitle = paste(sum(feature.drop), "cell(s) to the left of cutoff have too few detected genes"))

as.data.frame(colData(cdSc)) %>% cbind(., feature.drop) %>%
    rename(LowDetected = feature.drop) %>%
    ggplot(aes(x = factor(Sample), y = detected)) + geom_violin(size = 1) +
    geom_jitter(aes(color = LowDetected), alpha = 0.3, size = 0.1, 
                position = position_jitter(height = 0, width = 0.15, seed = 123)) +
    theme_classic(base_size = 20) + scale_color_manual(values = c("black", "red")) + 
    guides(color = guide_legend(override.aes = list(size = 4, alpha = 1))) +
    labs(x = "Sample", y = "Number of detected genes", title = "Total detected genes", fill = "Sample",
         subtitle = paste(sum(feature.drop), "cell(s) coloured in red have too few detected genes"))

## Mitochondrial proportion

In [ ]:
# Change this setting to adjust and show different threshold
mad3 <- 3
mito.drop <- isOutlier(cdSc$subsets_Mt_percent, batch = cdSc$Sample, nmads = mad3, type = "higher")
cut_off_MT <- attr(mito.drop, "thresholds")["higher",]
cut_off_MT <- cut_off_MT[levels(cdSc$Sample)] # reorder to levels
round(cut_off_MT,2)

# Set minimum cutoff at 10% if the MAD cutoff is lower than 10%
cut_off_MT <- ifelse(cut_off_MT < 10, 10, cut_off_MT)
# Set maximum cutoff at 15% if the MAD cutoff is higher than 15%
cut_off_MT <- ifelse(cut_off_MT > 15, 15, cut_off_MT)
round(cut_off_MT,2)

mito.drop <- c()
for(i in 1:length(cut_off_MT)) {
    mito.drop <- c(mito.drop, cdSc$subsets_Mt_percent[cdSc$Sample == names(cut_off_MT[i])] > cut_off_MT[i])
}

p <- as.data.frame(colData(cdSc)) %>% 
    left_join(data.frame(thresholds = cut_off_MT, Sample = names(cut_off_MT))) %>% 
    ggplot(aes(x = subsets_Mt_percent, fill = factor(Sample))) + 
    geom_density(color = NA, alpha = 0.5) + facet_wrap(~ Sample) +
    geom_vline(aes(xintercept = thresholds), colour = "red", linetype = "longdash") +
    geom_text(aes(x = thresholds, label = round(thresholds, 2)), y = Inf, vjust = 2, hjust = -0.2, size = 5) +
    theme_classic(base_size = 20) + scale_fill_manual(values = c_sample_col) +
    theme(legend.position = "none") +
    labs(x = "Mitochondrial proportion", title = "Mitochondrial proportion", fill = "Sample", 
         subtitle = paste(sum(mito.drop), "cell(s) to the right of cutoff have too high mitochondrial proportion"))

# Visual zoom to between 0 - 50%
if(max(cdSc$subsets_Mt_percent) > 50) {
    p <- p + coord_cartesian(xlim = c(0, 50))
}

p

## Novelty (complexity of RNA species)

The expected novelty is about 0.8.

In [ ]:
# Change this setting to adjust and show different threshold
cut_off_novelty = 0.8
novelty.drop <- cdSc$log10GenesPerUMI < cut_off_novelty

as.data.frame(colData(cdSc)) %>%
    ggplot(aes(x = log10GenesPerUMI, fill = factor(Sample))) + 
    geom_density(color = NA, alpha = 0.5) + facet_wrap(~ Sample) +
    geom_vline(xintercept = cut_off_novelty, colour = "red", linetype = "longdash") +
    annotate("text", x = cut_off_novelty, y = Inf, label = cut_off_novelty, 
             vjust = 2, hjust = -0.2, size = 5) +
    theme_classic(base_size = 20) + scale_fill_manual(values = c_sample_col) +
    theme(legend.position = "none") +
    labs(x = "log10 nGene per log10 nUMI", title = "Complexity of RNA species", fill = "Sample", 
         subtitle = paste(sum(novelty.drop), "cells to the left of cutoff have too low complexity"))

## Show summary

In [ ]:
print(paste0("Cells removed if: [",
           paste(names(cut_off_reads), collapse = " / "), "]"))
print(paste("Read count below library size cutoff:", 
            paste(round(cut_off_reads, 2), collapse = " / ")))
print(paste("Number of genes expressed below feature cutoff:", 
            paste(round(cut_off_genes, 2), collapse = " / ")))
print(paste("MT percent above mito cutoff:", 
            paste(round(cut_off_MT, 2), collapse = " / ")))
print(paste("Novelty cutoff:", round(cut_off_novelty, 4)))

In [ ]:
# Discard summary
discard <- libsize.drop | feature.drop | mito.drop | novelty.drop
cdSc$discard = discard

venn.df <- data.frame(Sample = cdSc$Sample, LibSize = libsize.drop, FeaturesExp = feature.drop, 
                      MitoProp = mito.drop, Novelty = novelty.drop, Total = discard)

print(paste("Total number of cells removed:"))
DataFrame(Cells = colSums(venn.df[,2:6]))

print(paste("Per-sample breakdown:"))
venn.df %>% group_by(Sample) %>% summarise_all(sum) %>% DataFrame

In [ ]:
limma::vennDiagram(venn.df[,2:5], cex = c(1.5,1.2,1.0))

Use __QC plot(s) below__ to check result and decide whether further filtering is required.

In [ ]:
p <- plotColData(cdSc, x = "sum", y = "detected", colour_by = "subsets_Mt_percent", 
                 other_fields = c("Sample","discard"), point_size = 1, point_alpha = 0.3, theme_size = 20) +
    facet_grid(discard ~ Sample) +
    scale_x_log10(breaks = log10_breaks, labels = log10_labels) +
    scale_y_log10(breaks = log10_breaks, labels = log10_labels) +
    guides(color = guide_colourbar(title = "Mitochondrial\nproportion (%)")) +
    labs(x = "Library size", y = "Number of detected genes")

fig(width = 16, height = 8, res = 120)
oldScaterPlotFix(p)
reset.fig()

## Filter poor-quality cells

A new object is created with the filtered dataset.

In [ ]:
cdScFilt <- cdSc[, !discard]
cdScFilt

In [ ]:
print(paste("Total cells before quality filtering =", ncol(cdSc)))
print(paste("Total cells before quality filtering, per sample:"))
table(colData(cdSc)$Sample)

print(paste("Total cells remaining after quality filtering =",ncol(cdScFilt)))
print(paste("Total cells after quality filtering, per sample:"))
table(colData(cdScFilt)$Sample)

## Show cells with extremely high counts

Use `isOutlier` and MAD cutoff to investigate possible outlier cells with high read-depth (could be doublets).

In [ ]:
# Change this setting to adjust and show different threshold
mad4 <- 5
highcount.drop <- isOutlier(cdScFilt$sum, nmads = mad4, batch = cdScFilt$Sample, type = "higher", log = TRUE)
cut_off_highcount <- attr(highcount.drop, "thresholds")["higher",]
cut_off_highcount <- cut_off_highcount[levels(cdSc$Sample)] # reorder to levels
round(cut_off_highcount,2)

# Apply same cutoff to all samples
cut_off_highcount <- 75000
highcount.drop <- cdScFilt$sum > cut_off_highcount
cut_off_highcount <- setNames(rep(cut_off_highcount, length(levels(cdScFilt$Sample))), levels(cdScFilt$Sample))
round(cut_off_highcount,2)

as.data.frame(colData(cdScFilt)) %>% 
    left_join(data.frame(thresholds = cut_off_highcount, Sample = names(cut_off_highcount))) %>%
    ggplot(aes(x = sum, fill = factor(Sample))) +
    geom_density(color = NA, alpha = 0.5) + facet_wrap(~ Sample) +
    geom_vline(aes(xintercept = thresholds), colour = "red", linetype = "longdash") +
    geom_text(aes(x = thresholds, label = round(thresholds, 2)), y = Inf, vjust = 2, hjust = 1.2, size = 5) +
    theme_classic(base_size = 20, base_family = "mono") + scale_fill_manual(values = c_sample_col) +
    scale_x_log10(breaks = log10_breaks, labels = log10_labels) +
    theme(legend.position = "none") +
    labs(x = "Library Size", title = "Total count", fill = "Sample", 
         subtitle = paste(sum(highcount.drop), "cell(s) to the right of cutoff have too high UMI count"))

as.data.frame(colData(cdScFilt)) %>% cbind(., highcount.drop) %>%
    rename(HighCount = highcount.drop) %>%
    ggplot(aes(x = factor(Sample), y = sum)) + geom_violin(size = 1) +
    geom_jitter(aes(color = HighCount, size = HighCount, alpha = HighCount), 
                position = position_jitter(height = 0, width = 0.15, seed = 123)) +
    scale_alpha_discrete(range = c(0.5, 1)) + scale_size_discrete(range = c(1, 2)) +
    theme_classic(base_size = 20) + scale_color_manual(values = c("black", "red")) +
    scale_y_continuous(labels = unit_format(unit = "K", scale = 1e-3)) +
    guides(color = guide_legend(override.aes = list(size = 4, alpha = 1))) +
    labs(x = "Sample", y = "Library Size", title = "Total count", fill = "Sample",
         subtitle = paste(sum(highcount.drop), "cell(s) coloured in red have extremely high UMIs"))

Show the number of cells defined as extremely high counts (`HighCount = TRUE`).

In [ ]:
table(Sample = cdScFilt$Sample, HighCount = highcount.drop)

## Filter cells with high counts

This changes `cdsFilt` permanently. Make a temporary object if unsure, or re-create `cdsFilt` from `cdSc` from previous steps. We also re-calculatingthe PerFeatureQC as the number of cells have changed.

<div class="alert alert-warning">
  <strong>Warning!</strong> For aggregate results, additional filtering may be required on a per-sample basis.
</div>

In [ ]:
cdScFilt <- cdScFilt[, !highcount.drop]

In [ ]:
as.data.frame(colData(cdScFilt)) %>% 
    ggplot(aes(x = factor(Sample), y = sum, color = Sample)) + geom_violin(size = 1) +
    geom_jitter(alpha = 0.5, size = 1, position = position_jitter(height = 0, width = 0.15, seed = 123)) +
    theme_classic(base_size = 20) + scale_color_manual(values = c_sample_col) +
    guides(color = guide_legend(override.aes = list(size = 4, alpha = 1))) +
    theme(legend.position = "none") + labs(x = "Sample", y = "Library Size", title = "Total count")

In [ ]:
# Remove cells from specific samples?
#cdScFilt <- cdScFilt[,!((cdScFilt$Sample %in% '12_W') & (cdScFilt$sum > 50000))]
#cdScFilt <- cdScFilt[,!((cdScFilt$Sample %in% '8_W') & (cdScFilt$sum > 40000))]

## Update PerFeatureQC due to cell filtering

In [ ]:
rowData(cdScFilt) = rowData(cdScFilt)[,c("ID","Symbol","Type","SEQNAME","is_mito","is_ambient")]
cdScFilt <- addPerFeatureQC(cdScFilt, detection_limit = 0, BPPARAM = bpp)
rowData(cdScFilt)$n_cells_by_counts <- rowData(cdScFilt)$detected/100 * ncol(cdScFilt)
rowData(cdScFilt)$pct_dropout <- 100 - rowData(cdScFilt)$detected

# This normalises after QC filtering.
# Use default size.factors for now, we will calculate later
logcounts(cdScFilt) <- log2(calculateCPM(cdScFilt) + 1)

cdScFilt

In [ ]:
print(paste0("Total cells remaining after filtering high read-depth cells = ", ncol(cdScFilt)))
print(paste0('Total cells remaining after filtering for each sample:'))
table(colData(cdScFilt)$Sample)

## Save image

In [ ]:
save.image("data.RData")

# 4. Classification of cell cycle phase<a name='section4' />

The prediction method used here is described by [Scialdone et al. (2015)](http://www.sciencedirect.com/science/article/pii/S1046202315300098) to classify cells into cell cycle phases based on the gene expression data. Using a training dataset, the sign of the difference in expression between two genes was computed for each pair of genes. Pairs with changes in the sign across cell cycle phases were chosen as markers. Cells in a test dataset can then be classified into the appropriate phase, based on whether the observed sign for each marker pair is consistent with one phase or another. We do the cell cycle classification before gene filtering as this provides more precise cell cycle phase classifications. This approach is implemented in the Cyclone function using a pre-trained set of marker pairs for human data. Some additional work is necessary to match the gene symbols in the data to the Ensembl annotation in the pre-trained marker set.



Previously, the workflow uses `org.XX.eg.db` to map SYMBOL from `rowData` to Ensembl ID. However the `rowData` now contains Ensembl ID, therefore the mapping step is not necessary.

<div class="alert alert-warning">
  <strong>Warning!</strong> Edits required to choose human or mouse cycle markers.
</div>

In [ ]:
# Human
sample.pairs <- readRDS(system.file("exdata", "human_cycle_markers.rds", package = "scran"))

# Mouse
#sample.pairs <- readRDS(system.file("exdata", "mouse_cycle_markers.rds", package = "scran"))

set.seed(12345)
system.time({ 
    assignments <- cyclone(cdScFilt, sample.pairs, gene.names = rowData(cdScFilt)$ID, verbose = TRUE, 
                           BPPARAM = bpp) 
})

In [ ]:
fig(width = 12, height = 10)
data.frame(x = assignments$score$G1, y = assignments$score$G2M,
           Phases = assignments$phases, Sample = colData(cdScFilt)$Sample) %>%
    ggplot(aes(x = x,y = y, shape = Phases, color = Sample)) +
    geom_point(size = 2, alpha = 0.5) + theme_classic(base_size = 20) + 
    facet_wrap(~ Sample) +
    scale_x_continuous("G1 score", limits = c(0, 1)) +
    scale_y_continuous("G2/M score", limits = c(0, 1)) +
    scale_color_manual(values = c_sample_col) +
    scale_shape_manual(values = c("G1" = 15, "G2M" = 17, "S" = 19)) +
    guides(color = guide_legend(override.aes = list(size = 4, alpha = 1)), 
           shape = guide_legend(override.aes = list(size = 4))) +
    geom_segment(aes(x = 1/2, y = 0, xend = 1/2, yend = 1/2), colour = "black") +
    geom_segment(aes(x = 0, y = 1/2, xend = 1/2, yend = 1/2), colour = "black") +
    geom_segment(aes(x = 1/2, y = 1/2, xend = 1, yend = 1), colour = "black") +
    annotate("label", x = 0.25, y = 0.05, size = 8, fill = "white", alpha = 0.5, label = "S") +
    annotate("label", x = 0.90, y = 0.40, size = 8, fill = "white", alpha = 0.5, label = "G1") +
    annotate("label", x = 0.50, y = 0.98, size = 8, fill = "white", alpha = 0.5, label = "G2/M") +
    ggtitle("Cell cycle phase scores")
reset.fig()

In [ ]:
fig(width = 8, height = 8)
smoothScatter(assignments$score$G1, assignments$score$G2M, xlab = "G1 score", ylab = "G2/M score", 
              pch = 19, cex = 0.6, nbin = 1920)
reset.fig()

__Background about Cell Cycle Analysis__

- Cells are classified as being in G1 phase (not in cell division aka cell cycle) if their G1 score is above 0.5 and greater than the G2/M score.
- Cells are classified as being in S phase (synthesis of DNA, replication) if neither score is above 0.5.
- Cells are classified as being in G2/M phase (gap between DNA synthesis and mitosis) if their G2/M score is above 0.5 and greater than the G1 score.
 
The cell-cycle status of cells can be a significant confounding factor in some datasets i.e. clusters forming on the basis of cell cycle status instead of other biological factors of interest. The goal at this stage is only to assess the cell cycle status of cells not to try normalise it away.

This method would be less accurate for data that are substantially different from those used in the training set, e.g., due to the use of a different protocol. This dataset uses UMI counts, which has an entirely different set of biases, e.g., 3’-end coverage only, no length bias, no amplification noise. These new biases (and the absence of expected biases) may interfere with accurate classification of some cells. So there is some uncertainty with this analysis. 

Nevertheless we need to keep in mind that there could be quite high cell-cycle effect which might confound the dataset. To avoid problems from misclassification, no processing of this dataset by cell cycle phase will be done here.

In [ ]:
print("Predicted phase:")
table(Sample = colData(cdScFilt)$Sample, Phases = assignments$phases)

Generally if they are on G1 stage, then they are not in cell-cycle stage and if on G2/M then they are cell-cycle stages. If they are in S, which is the synthesis phase indicating DNA is being replicated.

<div class="alert alert-info">
  <strong>About this dataset: </strong> (edits required)
  <ul>
    <li>It looks like majority of the cells have a high G2/M score.</li>
    <li>This indicates that the cells are going through cell-cycle stages.</li>
  </ul>
</div>

In [ ]:
## Assigning cell-cycle stages to the scater object
colData(cdScFilt)$CellCycle <- factor(assignments$phases)
cdScFilt

In [ ]:
save.image("data.RData")

# 5. Marking low-abundance genes<a name='section5' />

The goal is to mark genes that will be good features to discriminate the cells. 

Low-abundance genes are problematic as zero or near-zero counts do not contain enough information for reliable statistical inference ([Bourgon et al., 2010](http://www.pnas.org/content/107/21/9546)). In addition, the discreteness of the counts may interfere with downstream statistical procedures, e.g., by compromising the accuracy of continuous approximations. 

The more low frequency, low expression genes the more noise. 

These genes are likely to be dominated by drop-out events ([Brennecke et al., 2013](https://www.nature.com/articles/nmeth.2645)), which limits their usefulness in later analyses. Removal of these genes improves discreteness and reduces the amount of computational work without major loss of information.

Generally in scRNA-seq low-abundance genes are defined as those with an average count below a filter threshold of 1. But 10X Chromium is based on UMI counts, which are lower but better quality counts, so setting the threshold to 1 would filter a large number of cells unfairly. 

In the analysis below we go through an iterative process using the figure below, starting with 0.01 to 0.06 to assess what cutoff to use. To check whether the chosen threshold is suitable, we examine the distribution of log-means across all genes (Figure below). Generally for higher number of cells there is a peak on the right hand side that represents the bulk of moderately expressed genes while in the middle there is a rectangular component that corresponds to lowly expressed genes. The filter threshold should cut the distribution at some point along the rectangular component to remove the majority of low-abundance genes. As the blue line repsresents in the figure below, it cuts the counts at the rectangular component. Generally 9,000 - 16,000 genes is good (mouse) and 7,500-16,000 for human.

## Evaluate expression distribution

In [ ]:
# Identify genes that are not expressed in any cell
not.expressed = rowData(cdScFilt)$n_cells_by_counts == 0
table(not.expressed)

# Identify low-abundance genes
n = 6
cutoff_labels <- LETTERS[1:n]
cutoff_values <- c(0.005, seq(0.01, 0.01*(n-1), by = 0.01))

# Exclude genes that are not expressed at all
# Prevents `ggplot2` warnings about *non-finite values* when transformed to log scale, without affect the calculation.
ave.counts <- data.frame(Mean = rowData(cdScFilt)$mean[!not.expressed])
rownames(ave.counts) = rownames(rowData(cdScFilt))[!not.expressed]

for(i in 1:n) {
        ave.counts[, ncol(ave.counts) + 1] <- ave.counts$Mean >= cutoff_values[i]
        ave.counts.lab <- paste0(cutoff_labels[i], " (>=", cutoff_values[i], "; ", sum(ave.counts$Mean >= cutoff_values[i])," genes)")
        names(ave.counts)[ncol(ave.counts)] <- ave.counts.lab

        print(paste("Mean count >=", cutoff_values[i], "leaves", sum(ave.counts$Mean >= cutoff_values[i]), "genes"))
}

The threshold should be closer to the start or middle of the rectangular component.

In [ ]:
fig(width = 16, height = 10)
# Plot 1
ave.counts %>% gather(Cutoff, Pass, -Mean) %>%
        ggplot(aes(x = Mean, fill = Pass)) +
        geom_histogram(color = "white", size = 0.3, bins = 100) +
        scale_fill_manual(values = c("red", "blue")) +
        scale_x_log10(breaks = log10_breaks, labels = log10_labels) +
        facet_wrap(~ Cutoff, ncol = 2) + theme_classic(base_size = 20) +
        theme(legend.position = "top")
reset.fig()

cutoff <- data.frame(labels = colnames(ave.counts)[2:ncol(ave.counts)], values = cutoff_values)

fig(width = 16, height = 7)
# Plot 2
ave.counts %>% ggplot(aes(x = Mean)) +
        geom_histogram(color = "white", fill = "black", size = 0.3, bins = 100) +
        scale_x_log10(breaks = log10_breaks, labels = log10_labels) +
        theme_classic(base_size = 20) +
        geom_vline(mapping = aes(xintercept = values, colour = labels), size = 1,
                   data = cutoff) +
        guides(color = guide_legend(nrow = 3)) +
        theme(legend.position = "top")
reset.fig()

## Show top most-expressed genes

__Check identities of the most highly expressed genes__. This should generally be dominated by constitutively expressed transcripts, such as those for ribosomal or mitochondrial proteins. The presence of other classes of features may be cause for concern if they are not consistent with expected biology. For example, the absence of ribosomal proteins and/or the presence of their pseudogenes are indicative of suboptimal alignment.

In [ ]:
# Number of genes to show
topexp = 50

# Print gene symbols
head(rownames(ave.counts[order(ave.counts$Mean, decreasing = TRUE),]), topexp)

Plot below shows percentage of total counts (per cell) assigned to the top 50 most highly-abundant genes in the dataset.

Each row in the plot below corresponds to a gene; each bar corresponds to the expression of a gene in a single cell; and the circle indicates the median expression of each gene, with which genes are sorted. Bars are coloured by the total number of expressed genes in each cell.

In [ ]:
fig(width = 14, height = 12)
plotHighestExprs(cdScFilt, n = topexp, exprs_values = "counts", colour_cells_by = "detected") +
    guides(color = guide_colourbar(title = "Detected genes")) + theme_bw(base_size = 20)
reset.fig()

<div class="alert alert-warning">
  <strong>Warning!</strong> Do the top N most genes appear to agree with the biology?
</div>

This mean-based filter tends to be less aggressive. A gene will be retained as long as it has sufficient expression in any subset of cells. Genes expressed in fewer cells require higher levels of expression in those cells to be retained, but this is not undesirable as it avoids selecting uninformative genes (with low expression in few cells) that contribute little to downstream analyses, e.g., HVG detection or clustering. In contrast, the “at least n” filter depends heavily on the choice of n. With n = 10, a gene expressed in a subset of 9 cells would be filtered out, regardless of the level of expression in those cells. This may result in the failure to detect rare subpopulations that are present at frequencies below n. While the mean-based filter will retain more outlier-driven genes, this can be handled by choosing methods that are robust to outliers in the downstream analyses.
Thus, we apply the mean-based filter to the data by subsetting the SCESet object as shown below. 

## Mark low-abundance genes

Instead of remove the low abundance genes from the `sce` object, we will mark these genes with "`is_pass = FALSE`".

<div class="alert alert-info">
    <strong>Info!</strong> Decide cutoff and create a new object <code>cdScFiltAnnot</code>.
</div>

In [ ]:
# Decide cutoff
is.pass <- rowData(cdScFilt)$mean >= 0.02

# Mark genes that are not low abundance
cdScFiltAnnot <- cdScFilt
rowData(cdScFiltAnnot)$is_pass <- is.pass
table(rowData(cdScFiltAnnot)$is_pass)

### Frequency of expression (i.e., number of cells with non-zero expression) against the mean

These two metrics should be positively correlated with each other for most genes. This plot can be useful to assess the effects of technical dropout in the dataset. We fit a smoothing curve (selected by `ggplot2`)<sup>*</sup> for the relationship between expression frequency and mean expression. We use this curve to define the number of genes above high technical dropout and the numbers of genes that are expressed in at least 50% and at least 25% of cells.

Previously we would use the `plotExprsFreqVsMean` function to generate this plot, however it is now deprecated and does not support the current QC metric.

<sup>*</sup> The smoothing method is chosen based on the size of the largest group. `stats::loess()` is used for less than 1,000 observations; otherwise `mgcv::gam()` is used.

In [ ]:
# Calculate log2 mean.
log2means <- log2(rowData(cdScFiltAnnot)$mean + 1)

# data frame with expression mean and frequency.
mn_vs_fq <- data.frame(mn = log2means, fq = rowData(cdScFiltAnnot)$detected / 100)
text_x_loc <- min(mn_vs_fq$mn) + 0.7 * diff(range(mn_vs_fq$mn))

p <- plotRowData(cdScFiltAnnot, x = data.frame(X = log2means, check.names = FALSE), 
                 y = data.frame(Y = rowData(cdScFiltAnnot)$detected, check.names = FALSE), 
                 colour_by = "pct_dropout", theme_size = 20) + 
    geom_smooth(aes_string(x = "mn", y = "100*fq"), data = mn_vs_fq, colour = "firebrick", size = 1, se = TRUE) + 
    geom_hline(yintercept = 50, linetype = 2) + # 50% dropout 
    guides(color = guide_colourbar(title = "Dropouts (%)")) + 
    scale_y_continuous(limits = c(0, 110), breaks = c(seq(0, 100, by = 25))) +
    geom_text_repel(aes(label = ifelse(X > quantile(log2means, 0.999) & 
                                       Y > quantile(rowData(cdScFiltAnnot)$detected, 0.999), 
                                       names(cdScFiltAnnot), "")), color = "black", seed = 123,
                    min.segment.length = unit(0, "lines"), box.padding = 0.5, size = 4.5, max.overlaps = Inf) +
    labs(x = "log2 Mean expression", y = "Percentage of expressing cells") +
    annotate("text", x = text_x_loc, y = 40, size = 6, 
             label = paste0(sum(mn_vs_fq$fq >= 0.5), " genes are expressed\nin at least 50% of cells")) + 
    annotate("text", x = text_x_loc, y = 20, size = 6, 
             label = paste0(sum(mn_vs_fq$fq >= 0.25), " genes are expressed\nin at least 25% of cells"))

fig(width = 16, height = 8)
oldScaterPlotFix(p)
reset.fig()

<div class="alert alert-info">
  <strong>Info!</strong> Preferably at least 1000 genes will be expressed in 50% of the cells. These uniformly expressed genes are not helpful in disciminating cells, but if the number of genes expressed in >50% is low, that means the dataset quality is not great because there are too many dropouts.
</div>

### Extreme highly expressed genes

Usually this is ***MALAT1***. Such genes can caused a skew in the smoothing trend line, and can also affect the mean-variance trend fitting by `modelGeneVar` during feature selection, i.e. highly variable genes (HVG) selection.

Below, we use a the mean-variance plot to observe the locations of the extreme highly expressed genes (99.99th quantile).

In [ ]:
logcpm.mean <- rowMeans(logcounts(cdScFiltAnnot))
logcpm.var <- rowVars(logcounts(cdScFiltAnnot))

p <- plotRowData(cdScFiltAnnot, x = data.frame(X = logcpm.mean, check.names = FALSE), 
                 y = data.frame(Y = logcpm.var, check.names = FALSE), 
                 colour_by = "pct_dropout", theme_size = 20) + 
    guides(color = guide_colourbar(title = "Dropouts (%)")) + 
    geom_text_repel(aes(label = ifelse(X > quantile(logcpm.mean, 0.9999) & 
                                       Y < 5, names(cdScFiltAnnot), "")), color = "black", seed = 123,
                    min.segment.length = unit(0, "lines"), box.padding = 0.5, size = 4.5) +
    labs(x = "log2 Mean expression", y = "Variance of log-counts")

fig(width = 16, height = 8)
oldScaterPlotFix(p)
reset.fig()

In [ ]:
# Show top 10 expressed genes
data.frame(Symbol = rowData(cdScFiltAnnot)$Symbol, log2means = log2means, 
           Detected = rowData(cdScFiltAnnot)$detected,
           log2CPM.mean = logcpm.mean, log2CPM.var = logcpm.var, 
           stringsAsFactors = FALSE) %>% arrange(desc(log2CPM.mean)) %>% head(10)

<div class="alert alert-warning">
  <strong>Warning!</strong> Are there any genes that have extremely high average expression, expressed in majority of the cells and low in variance?
</div>

#### Mark additional genes that are highly expressed, expressed in every cells and low in variance?

In [ ]:
highexp.genes <- c("MALAT1")
rowData(cdScFiltAnnot)[rowData(cdScFiltAnnot)$Symbol %in% highexp.genes,]$is_pass <- FALSE

# How many genes are marked is_pass
table(rowData(cdScFiltAnnot)$is_pass)

cdScFiltAnnot

Re-calculatingthe PerCellQC to add another subset to calculate only the `is_pass` genes.

In [ ]:
is.mito2 <- rowData(cdScFiltAnnot)$SEQNAME == "MT"
is.pass2 <- rowData(cdScFiltAnnot)$is_pass

colData(cdScFiltAnnot) = colData(cdScFiltAnnot)[,c("Sample","Barcode","CellCycle")]
if(packageVersion("scater") == "1.16.2") {
    cdScFiltAnnot <- addPerCellQC(cdScFiltAnnot, 
                                  subsets = list(Mt = is.mito2, Pass = is.pass2), # Add "Pass" subset
                                  BPPARAM = bpp)
} else {
    cdScFiltAnnot <- addPerCellQC(cdScFiltAnnot, percent.top = c(50, 100, 200, 500), 
                                  subsets = list(Mt = is.mito2, Pass = is.pass2), # Add "Pass" subset
                                  BPPARAM = bpp)
}

colData(cdScFiltAnnot)$log10Sum <- log10(colData(cdScFiltAnnot)$sum+1)
colData(cdScFiltAnnot)$subsets_Pass_log10GenesPerUMI <- log10(colData(cdScFiltAnnot)$subsets_Pass_detected+1) / 
                                                        log10(colData(cdScFiltAnnot)$subsets_Pass_sum+1)
head(colData(cdScFiltAnnot), 5)

In [ ]:
p <- plotRowData(cdScFiltAnnot, x = "mean", y = "detected", colour_by = "is_pass", theme_size = 20) +
    scale_x_log10(breaks = log10_breaks, labels = log10_labels) +
    scale_color_manual(values = c("cyan","red")) +
    guides(color = guide_legend(title = "Passed Gene", override.aes = list(size = 4, alpha = 1))) +
    labs(x = "Mean counts across all cells", y = "Proportion of expressing cell (%)")

oldScaterPlotFix(p)

In [ ]:
limma::vennDiagram(data.frame(Is_Mito = rowData(cdScFiltAnnot)$is_mito, 
                              Is_Ambient = rowData(cdScFiltAnnot)$is_ambient, 
                              Is_Pass = rowData(cdScFiltAnnot)$is_pass))

In [ ]:
save.image("data.RData")

# 6. Normalisation of read counts<a name='section6' />
Single cell RNA-seq data requires different normalisation to bulk data methods (e.g. DESeq2) because scRNA-seq is very sparse. Here the deconvolution based method will be used.

__Further detail on the deconvolution method to deal with zero counts:__ 
Read counts are subject to differences in capture efficiency and sequencing depth between cells ([Stegle et al., 2015](https://www.nature.com/articles/nrg3833)). Normalisation is required to eliminate these cell-specific biases prior to downstream quantitative analyses. In bulk data this is often done by assuming that most genes are not differentially expressed (DE) between cells. Any systematic difference in count size across the non-DE majority of genes between two cells is assumed to represent bias and is removed by scaling. More specifically, “size factors” are calculated that represent the extent to which counts should be scaled in each library. Single-cell data can be problematic due to the dominance of low and zero counts. To overcome this, counts from many cells are pooled to increase the count size for accurate size factor estimation ([Lun et al., 2016](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-0947-7)). Pool-based size factors are then “deconvolved” into cell-based factors for cell-specific normalisation.

Syed's old message about __pool size__ and __negative size-factors__: 

If pool size is too big then in some datasets an error will be called ("estimated negative size factor"). In that case go down to smaller pool sizes (e.g. 100, 150, 200, 250, 300). If still not low enough keep going reducing the size (50,100,150,200,300,400). But this can also give "estimated negative size factor". Adjustments to these step sizes can be tried if the tSNE plots indicate that library size is dominating cluster selection.

As there are some negative size-factors for this dataset, obviously it indicates there are low quality cells with few expressed genes, such that most genes still have zero counts even after pooling. They may also occur if insufficient filtering of low-abundance genes was performed.

To avoid negative size factors, the best solution is to increase the stringency of the filtering.

- If only a few negative size factors are present, they are likely to correspond to a few low-quality cells with few expressed genes. Such cells are difficult to normalise reliably under any approach, and can be removed by increasing the stringency of the quality control.

- If many negative size factors are present, it is probably due to insufficient filtering of low-abundance genes. This results in many zero counts and pooled size factors of zero, and can be fixed by filtering out more genes with a higher min.mean - see “Gene selection” in vignette.

Another approach is to increase in the number of sizes to improve the precision of the estimates. This reduces the chance of obtaining negative size factors due to estimation error, for cells where the true size factors are very small.

As there are quite a lof of negative size factors, prorablby due to insuffient filtewring of low-abundance genes. So I will set the `min.mean` value to higher values to calculate the size factor.

Since it is still producing negative size factors, probably there are cells with high ambient RNAs. 

## Compute deconvolution size factors

In general, it is more appropriate to pool more similar cells to avoid violating the assumption of a non-DE majority of genes. This can be done by specifying the `clusters` argument where cells in each cluster have similar expression profiles. Deconvolution is subsequently applied on the cells within each cluster, where there should be fewer DE genes between cells. A convenience function `quickCluster` is provided for this purpose, though any reasonable clustering can be used. Only a rough clustering is required here, as `computeSumFactors` is robust to a moderate level of DE within each cluster.

After performing `computeSumFactors`, size factors are stored in a new column called `sizeFactor`. The values can be retrieved by using the `sizeFactors` function.

In [ ]:
# Using pre-clustering
# We want at least 100 cells per cluster, i.e. min.size = 100
set.seed(12345)
qclust <- quickCluster(cdScFiltAnnot, min.size = 100)
table(qclust)

cdScFiltAnnot <- computeSumFactors(cdScFiltAnnot, cluster = qclust)
summary(sizeFactors(cdScFiltAnnot))

In [ ]:
# New "sizeFactor" column 
head(colData(cdScFiltAnnot), 5)

In [ ]:
DF <- data.frame(VAR1 = cdScFiltAnnot$sum, VAR2 = cdScFiltAnnot$sizeFactor)
model <- lm(VAR2 ~ VAR1, DF)
summary(model)

### Size factor versus library size

If the size factors are tightly correlated with the library sizes for all cells this would suggests that the systematic differences between cells are primarily driven by differences in capture efficiency or sequencing depth. Any DE between cells would yield a non-linear trend between the total counts and size factor, and/or increased scatter around the trend.

In [ ]:
fig(width = 7, height = 7)
ggplot(DF, aes(x = VAR1, y = VAR2)) + geom_point(size = 1, alpha = 0.5) +
    scale_x_continuous(labels = unit_format(unit = "K", scale = 1e-3)) +    
    stat_smooth(method = lm) + theme_classic(base_size = 20) +
    annotate("text", x = ceiling(max(DF$VAR1))*0.25, y = quantile(DF$VAR2, 0.999), 
             label = paste0("italic(R)^2 ==", round(summary(model)$r.squared,4)), size = 7, parse = TRUE) +
    labs(x = "Library size", y = "Size factor")
reset.fig()

<div class="alert alert-info">
  <strong>About this dataset: </strong> (edits required)
  <ul>
    <li>Has high R-squared value.</li>
    <li>The size factors are very tightly correlated with the library size.</li>
  </ul>
</div>

## Compute normalised expression values using newly calculated size factors

The count data are used to compute normalised log-expression values for use in downstream analyses. Each value is defined as the log-ratio of each count to the size factor for the corresponding cell, after adding a prior count of 1 to avoid undefined values at zero counts. Division by the size factor ensures that any cell-specific biases are removed. If spike-in-specific size factors are present in sce, they will be automatically applied to normalise the spike-in transcripts separately from the endogenous genes.

The log-transformation provides some measure of variance stabilization ([Law et al., 2014](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4053721/)), so that high-abundance genes with large variances do not dominate downstream analyses. The computed values are stored as an expression matrix in addition to the other assay elements.

__Re-calculate the CPM normalisation using size factors__

We need to re-calculate the `counts-per-million` because large number of genes have been identified as no/low abundance and this would have impacted the library size and therefore how the CPM was calculated. Now, as the number of genes have been reduced, so is the library size. 

When `sizeFactor` is present in `colData()`, the normalised expression computation will use this by default when `size.factors = NULL`

In [ ]:
logcounts(cdScFiltAnnot) <- log2(calculateCPM(cdScFiltAnnot) + 1)
cdScFiltAnnot

In [ ]:
# Average logcounts distribution
summary(rowMeans(logcounts(cdScFiltAnnot)))

In [ ]:
save.image("data.RData")

## Checking for important technical factors

Check whether there are technical factors that contribute substantially to the heterogeneity of gene expression. If so, the factor may need to be regressed out to ensure that it does not inflate the variances or introduce spurious correlations. For a dataset with a simple experimental design there are no plate or batch effects to examine. However, there could be other technical factors like the cell-cycle effect or dropouts. The plots below show before and after normalisation.

*The default `exprs_values` in `plotExplanatoryVariables` is to use logcounts rather than raw counts.*

### Limit to `is_pass` genes

In [ ]:
# Raw counts (is_pass genes)
# Sample removed from variable list for single-sample dataset
plotExplanatoryVariables(cdScFiltAnnot[rowData(cdScFiltAnnot)$is_pass,], 
                         exprs_values = "counts", theme_size = 20,
                         variables = c("subsets_Pass_sum", "subsets_Pass_detected", 
                                       "subsets_Mt_percent", "CellCycle"))

In [ ]:
# log2 CPM (is_pass genes)
# Sample removed from variable list for single-sample dataset
plotExplanatoryVariables(cdScFiltAnnot[rowData(cdScFiltAnnot)$is_pass,], 
                         exprs_values = "logcounts", theme_size = 20,
                         variables = c("subsets_Pass_sum", "subsets_Pass_detected", 
                                       "subsets_Mt_percent", "CellCycle"))

### Include `Sample` variable in the plot

<div class="alert alert-danger">
  <strong>Danger!</strong> Only works for 2 or more samples.
</div>

In [ ]:
# Raw counts (is_pass genes)
plotExplanatoryVariables(cdScFiltAnnot[rowData(cdScFiltAnnot)$is_pass,], 
                         exprs_values = "counts", theme_size = 20,
                         variables = c("subsets_Pass_sum", "subsets_Pass_detected", 
                                       "subsets_Mt_percent", "CellCycle", "Sample"))

In [ ]:
# log2 CPM (is_pass genes)
plotExplanatoryVariables(cdScFiltAnnot[rowData(cdScFiltAnnot)$is_pass,], 
                         exprs_values = "logcounts", theme_size = 20,
                         variables = c("subsets_Pass_sum", "subsets_Pass_detected", 
                                       "subsets_Mt_percent", "CellCycle", "Sample"))

__Assessing this plot.__ Typically the number of total number of UMIs (sum) and number of detectable genes (detected) explain the major variability in scRNA-seq datasets. This indicates that cells vary significantly based on number of genes that they are expressing. 

Number of detectable genes explaining majority of the variability is a common issue in single-cell RNA-seq. This could be due to the sparsity of the data. The PCA plot might be separating cells just because they have very different number of expressed genes in total.

However, this would not confound the t-SNE plot which would take the non-linear relationship between the variables. As long as the peaks for `sum` and `detected` are below 10 this is normal.

In the cell-cycle plot shown earlier and this plot help establish the importance of the cell-cycle effect.

# 7. Identifying genes for feature selection (HVG)<a name='section7' />

The goal is to identify genes that will be good features to discriminate the cells. In this step, the dataset is filtered to keep only genes that are “informative” of the variability in the data. Highly Variable Genes (HVG) are the ones driving heterogeneity across the population of cells. 

__Detail__ HVG identification requires estimation of the variance in expression for each gene, followed by decomposition of the variance into biological and technical components. HVGs are then identified as those genes with the largest biological components. This avoids prioritizing genes that are highly variable due to technical factors, such as sampling noise during RNA capture and library preparation.

In the recent implementation of `seurat`, Rahul Satija took a slightly different approach for HVG calculation. They calculate the variance and mean for each gene in the dataset (storing this in `object@hvg.info`), and sorts genes by their variance/mean ratio (VMR). They have observed that for large-cell datasets with unique molecular identifiers, selecting highly variable genes (HVG) simply based on VMR is an efficient and robust strategy. 

But in the implementation used here a trend is fitted to the variance estimates of the endogenous genes, using the `use.spikes=FALSE` setting as shown below. This assumes that the majority of genes are not variably expressed, such that the technical component dominates the total variance for those genes. The fitted value of the trend is then used as an estimate of the technical component. 

__span:__ Low-abundance genes with mean log-expression below `min.mean` are not used in trend fitting, to preserve the sensitivity of span-based smoothers at moderate-to-high abundances. It also protects against discreteness, which can interfere with estimation of the variability of the variance estimates and accurate scaling of the trend. The default threshold is chosen based on the point at which discreteness is observed in variance estimates from Poisson-distributed counts. For heterogeneous droplet data, a lower threshold of 0.001-0.01 should be used.

#### About the returned DataFrame:

- mean: Mean normalised log-expression per gene.
- total: Variance of the normalised log-expression per gene.
- bio: Biological component of the variance.
- tech: Technical component of the variance.
- p.value, FDR: Raw and adjusted p-values for the test against the null hypothesis that bio<=0.

<div class="alert alert-info">
    <strong>Info!</strong> Create a new object <code>cdScFiltSub</code> that retain genes that are marked as <code>is_pass</code> and not <code>is_ambient</code>. Modelling is only performed on this subset of genes.
</div>

In [ ]:
cdScFiltSub <- cdScFiltAnnot[rowData(cdScFiltAnnot)$is_pass & !rowData(cdScFiltAnnot)$is_ambient,]
cdScFiltSub

## Model the variance of the log-expression profiles for each gene

<div class="alert alert-warning">
  <strong>Warning!</strong> We block on the donor of origin to mitigate batch effects during HVG selection.
</div>

In [ ]:
# Default assay.type is logcounts
set.seed(12345)
var.out <- modelGeneVar(cdScFiltSub, assay.type = "logcounts", block = cdScFiltSub$Sample, BPPARAM = bpp)
var.out[order(var.out$bio, decreasing = TRUE),]

## Model the per-gene count variance with Poisson noise

For each gene, the function computes the variance and mean of the log-expression values. A trend is fitted to the variance against the mean for simulated Poisson counts. The assumption is that the technical component is Poisson-distributed, or at least negative binomial-distributed with a known constant dispersion. This is useful for UMI count data sets that do not have spike-ins and are too heterogeneous to assume that most genes exhibit negligible biological variability.

<div class="alert alert-warning">
  <strong>Warning!</strong> We block on the donor of origin to mitigate batch effects during HVG selection.
</div>

*The normalised log-expression used to calculate the per-gene mean here are identical to that calculated by `logNormCounts`, and are different from that by log2 cpm method.*

In [ ]:
# Default assay.type is counts
set.seed(12345)
var.out2 <- modelGeneVarByPoisson(cdScFiltSub, assay.type = "counts", block = cdScFiltSub$Sample, BPPARAM = bpp)
var.out2[order(var.out2$bio, decreasing = TRUE),]

## Assess the fit

We assess the suitability of the trend fitted to the endogenous variances by examining whether it is consistent with the variances. The trend passes through or close to most of the endogenous gene variances, indicating that our assumption (that most genes have low levels of biological variability) is valid. This strategy exploits the large number of endogenous genes to obtain a stable trend. 

In [ ]:
# Function to create mean-vairance scatter plot 
VariableFeaturePlot <- function(sce = cdScFiltAnnot, var = var.out, hvg = NULL, title = NULL) {
    is_fail <- rownames(subset(rowData(sce), is_pass == FALSE))
    is_ambient <- rownames(subset(rowData(sce), is_ambient == TRUE))
    
    if(is.null(hvg)) {
        data <- as.data.frame(var) %>% rownames_to_column()
    } else {
        data <- as.data.frame(var) %>% rownames_to_column() %>% add_column(HVG = FALSE) %>%
        mutate(HVG = replace(HVG, rowname %in% hvg, TRUE))
    }

    data$status <- "Passed"
    if(sum(data$rowname %in% is_fail) > 0) {
        data[which(data$rowname %in% is_fail),]$status <- "Failed"
    }
    if(sum(data$rowname %in% is_ambient) > 0) {
        data[which(data$rowname %in% is_ambient),]$status <- "Ambient Contamination"
    }
    data$status = as.factor(data$status)
    
    plot <- ggplot(data, aes(x = mean, y = total, shape = status, alpha = status)) + 
    theme_classic(base_size = 18) + 
    scale_shape_manual(values = setNames(c(4, 1, 16), c("Failed", "Ambient Contamination", "Passed") )) +
    scale_alpha_manual(values = setNames(c(1, 1, 0.4), c("Failed", "Ambient Contamination", "Passed") )) +
    guides(shape = guide_legend(override.aes = list(size = 4, alpha = 1))) +
    labs(x = "Mean of log-expression", y = "Variance of log-expression", 
         shape = "Gene status", alpha = "Gene status")
                  
    if(is.null(hvg)) {
        plot <- plot + geom_point(size = 3) +
                # Label mean outliers (99.99th)
                geom_text_repel(aes(label = ifelse(mean > quantile(mean, 0.9999), rowname, "")), 
                                color = "black", alpha = 1, seed = 123, 
                                box.padding = 0.5, size = 4.5, max.overlaps = Inf)
    } else {
        plot <- plot + geom_point(aes(color = HVG), size = 3) + 
                scale_color_manual(labels = paste0(c('Non-HVG', 'HVG'), ':', table(data$HVG)), 
                                   values = c("black", "red")) +
                guides(color = guide_legend(override.aes = list(size = 4, alpha = 1))) +
                # Label top 10 HVGs
                geom_text_repel(aes(label = ifelse(rowname %in% head(hvg, 10), rowname, "")), 
                                color = "black", alpha = 1, seed = 123, 
                                box.padding = 0.5, size = 4.5, max.overlaps = Inf)
    }
    
    # Add trend line
    plot <- plot + stat_function(fun = function(x) metadata(var)$trend(x), 
                                 geom = "line", color = "gold", alpha = 1, size = 1.5)
    
    if(!is.null(title)) {
        plot <- plot + ggtitle(title)
    }
    
    plot
}

### Visualizing the fit

In [ ]:
# modelGeneVar
if("per.block" %in% colnames(var.out)) {
    blocked.stats <- var.out$per.block
    n <- length(names(blocked.stats))
    p <-  vector("list", n)

    for(i in 1:n) {
        p[[i]] <- VariableFeaturePlot(var = blocked.stats[[i]], 
                                      title = paste("modelGeneVar:", names(blocked.stats)[i])) +
            theme(legend.position = "top", legend.justification = "left")
    }
} else {
    p <-  vector("list", 1)
    p[[1]] <- VariableFeaturePlot(var = var.out, title = "modelGeneVar")
}

fig(width = 16, height = 9)
plot_grid(plotlist = p)
reset.fig()

Ideally the plot above would look like the mountain, where it would have rise in the middle but then drop off at the end (low variance for highly expressed genes). The trend line would follow it as well.

In [ ]:
# modelGeneVarByPoisson
if("per.block" %in% colnames(var.out2)) {
    blocked.stats <- var.out2$per.block
    n <- length(names(blocked.stats))
    p <-  vector("list", n)

    for(i in 1:n) {
        p[[i]] <- VariableFeaturePlot(var = blocked.stats[[i]], 
                                      title = paste("modelGeneVarByPoisson:", names(blocked.stats)[i])) +
            theme(legend.position = "top", legend.justification = "left")
    }
} else {
    p <-  vector("list", 1)
    p[[1]] <- VariableFeaturePlot(var = var.out2, title = "modelGeneVarByPoisson")
}

fig(width = 16, height = 9)
plot_grid(plotlist = p)
reset.fig()

## Select an appropriate set of highly variable genes

Once the per-gene variations are quantified, the next step is to select the HVGs to be used in downstream analyses. Selecting a larger number of HVGs will allow use to retain more potentially relevant genes but at the same time increasing noise from irrelevant genes that obscure interesting biological signal. There are several common methods used to guide HVG selection, which you can read more [here](http://bioconductor.org/books/release/OSCA/feature-selection.html#hvg-selection).

*Syed's old message about **Define HVGs**:
HVG are defined as genes with biological components that are significantly greater than zero at a false discovery rate (FDR) of 5%. These genes are interesting as they drive differences in the expression profiles between cells, and should be prioritized for further investigation. Here a gene is considered to be a HVG if it has a biological component greater than or equal to 0.5. For transformed expression values on the log2 scale, this means that the average difference in true expression between any two cells will be at least 2-fold. (This reasoning assumes that the true log-expression values are Normally distributed with variance of 0.5. The root-mean-square of the difference between two values is treated as the average log2-fold change between cells and is equal to unity.) The results are ranked by the biological component to focus on genes with larger biological variability. HVG can number anywhere between 200-5000 depending on the complexity of the dataset and depth of sequencing. [Ref: Current best practices in single‐cell RNA‐seq analysis: a tutorial](https://www.embopress.org/doi/10.15252/msb.20188746)*

### Select based on significance and biological components

In [ ]:
# "modelGeneVar" method
hvg.out <- as.data.frame(var.out) %>% dplyr::filter(FDR <= 0.05 & bio >= 0.5) %>%
        arrange(desc(bio)) %>% rownames
print(paste("Number of HVGs using FDR and Bio (modelGeneVar)", length(hvg.out)))
head(hvg.out, 20)

# "modelGeneVarByPoisson" method
hvg.out2 <- as.data.frame(var.out2) %>% dplyr::filter(FDR <= 0.05 & bio >= 0.5) %>%
        arrange(desc(bio)) %>% rownames
print(paste("Number of HVGs using FDR and Bio (modelGeneVarByPoisson)", length(hvg.out2)))
head(hvg.out2, 20)

### Select the top N% of genes with the highest biological components

**We select a larger number of HVGs to capture any batch-specific variation that might be present.**

In [ ]:
# "modelGeneVar" method
hvg.out <- getTopHVGs(var.out, prop = 0.30, var.field = "bio", var.threshold = 0)
print(paste("Number of HVGs using top 30% Bio (modelGeneVar)", length(hvg.out)))
head(hvg.out, 20)

# "modelGeneVarByPoisson" method
hvg.out2 <- getTopHVGs(var.out2, prop = 0.15, var.field = "bio", var.threshold = 0)
print(paste("Number of HVGs using top 15% Bio (modelGeneVarByPoisson)", length(hvg.out2)))
head(hvg.out2, 20)

### Visualise HVG selections

Below we mark the top N% HVG from both models in red.

In [ ]:
# modelGeneVar
if("per.block" %in% colnames(var.out)) {
    blocked.stats <- var.out$per.block
    n <- length(names(blocked.stats))
    p <-  vector("list", n)

    for(i in 1:n) {
        p[[i]] <- VariableFeaturePlot(var = blocked.stats[[i]], hvg = hvg.out,
                                      title = paste("modelGeneVar:", names(blocked.stats)[i])) +
            theme(legend.position = "none")
    }

    legend <- get_legend(p[[1]] + theme(legend.position = "top", legend.justification = "left"))
    p <- plot_grid(legend, plot_grid(plotlist = p), ncol = 1, rel_heights = c(0.1, 0.9))
} else {
    p <-  vector("list", 1)
    p[[1]] <- VariableFeaturePlot(var = var.out, hvg = hvg.out, title = "modelGeneVar")
    p <- plot_grid(plotlist = p)
}

fig(width = 16, height = 10)
p
reset.fig()

In [ ]:
# modelGeneVarByPoisson
if("per.block" %in% colnames(var.out2)) {
    blocked.stats <- var.out2$per.block
    n <- length(names(blocked.stats))
    p <-  vector("list", n)

    for(i in 1:n) {
        p[[i]] <- VariableFeaturePlot(var = blocked.stats[[i]], hvg = hvg.out2,
                                      title = paste("modelGeneVarByPoisson:", names(blocked.stats)[i])) +
            theme(legend.position = "none")
    }
    legend <- get_legend(p[[1]] + theme(legend.position = "top", legend.justification = "left"))
    p <- plot_grid(legend, plot_grid(plotlist = p), ncol = 1, rel_heights = c(0.1, 0.9))
} else {
    p <-  vector("list", 1)
    p[[1]] <- VariableFeaturePlot(var = var.out2, hvg = hvg.out2, title = "modelGeneVarByPoisson")
    p <- plot_grid(plotlist = p)
}

fig(width = 16, height = 10)
p
reset.fig()

Trends based purely on technical noise (i.e. `modelGeneVarByPoisson`) tend to yield large biological components for highly-expressed genes, including “house-keeping” genes coding for essential cellular components such as ribosomal proteins, which are considered uninteresting for characterizing cellular heterogeneity. These observations suggest that a more accurate noise model does not necessarily yield a better ranking of HVGs, though one should keep an open mind - house-keeping genes are regularly DE in a variety of conditions, and the fact that they have large biological components indicates that there is strong variation across cells that may not be completely irrelevant. [OSCA reference](http://bioconductor.org/books/release/OSCA/feature-selection.html#sec:spikeins)

<div class="alert alert-info">
  <strong>About this dataset: </strong> (edits required)
  <ul>
    <li>We use the the <code>getTopHVGs</code> method to choose HVGs from the <code>modelGeneVarByPoisson</code> model.</li>
    <li>We obtained a total of 1659 HVGs.</li>
  </ul>
</div>

## *For test purposes*

<div class="alert alert-warning">
  <strong>Warning!</strong> Here we are testing the proportion of ambient/failed genes selected as HVG if there is no pre-filtering performed on the input <code>sce</code>.
</div>

In [ ]:
# For test purposes - all genes (no subset)
set.seed(12345)
var.out3 <- modelGeneVarByPoisson(cdScFiltAnnot, assay.type = "counts", block = cdScFiltSub$Sample, BPPARAM = bpp)
hvg.out3 <- getTopHVGs(var.out3, prop = 0.1, var.field = "bio", var.threshold = 0)

df <- rowData(cdScFiltAnnot)[rownames(rowData(cdScFiltAnnot)) %in% hvg.out3,] %>% 
    as.data.frame() %>% arrange(Symbol)
table(Is_Pass = df$is_pass, Is_Ambient = df$is_ambient)

print("Failed genes identified as HVG:")
rownames(df[df$is_pass == FALSE,])

print("Genes affected by ambient contamination and identified as HVG:")
rownames(df[df$is_ambient == TRUE,])

In [ ]:
# Plot
title <- ggdraw() + draw_label("All genes", size = 22, fontface = "bold")
blocked.stats <- var.out3$per.block
n <- length(names(blocked.stats))

p <-  vector("list", n)
for(i in 1:n) {
    p[[i]] <- VariableFeaturePlot(var = blocked.stats[[i]], title = names(blocked.stats)[i]) + 
    theme(legend.position = "none")
}

legend <- get_legend(p[[1]] + theme(legend.position = "top", legend.justification = "left"))

fig(width = 16, height = 9)
plot_grid(title, legend, plot_grid(plotlist = p), ncol = 1, rel_heights = c(0.075, 0.075, 0.85))
reset.fig()

# Highlight HVGs
p <-  vector("list", n)
for(i in 1:n) {
    p[[i]] <- VariableFeaturePlot(var = blocked.stats[[i]], hvg = hvg.out3, title = names(blocked.stats)[i]) + 
    theme(legend.position = "none")
}

legend <- get_legend(p[[1]] + theme(legend.position = "top", legend.justification = "left"))

fig(width = 16, height = 9)
plot_grid(title, legend, plot_grid(plotlist = p), ncol = 1, rel_heights = c(0.075, 0.075, 0.85))
reset.fig()

In [ ]:
# For test purposes - All genes but exclude genes affected by ambient contamination
set.seed(12345)
var.out4 <- modelGeneVarByPoisson(cdScFiltAnnot[!rowData(cdScFiltAnnot)$is_ambient,], assay.type = "counts", 
                                  block = cdScFiltSub$Sample, BPPARAM = bpp)
hvg.out4 <- getTopHVGs(var.out4, prop = 0.1, var.field = "bio", var.threshold = 0)

df <- rowData(cdScFiltAnnot)[rownames(rowData(cdScFiltAnnot)) %in% hvg.out4,] %>% 
    as.data.frame() %>% arrange(Symbol)
table(Is_Pass = df$is_pass)

print("Failed genes identified as HVG:")
rownames(df[df$is_pass == FALSE,])

In [ ]:
# Plot
title <- ggdraw() + draw_label("All genes but exclude ambient contamination", size = 22, fontface = "bold")
blocked.stats <- var.out4$per.block
n <- length(names(blocked.stats))

p <-  vector("list", n)
for(i in 1:n) {
    p[[i]] <- VariableFeaturePlot(var = blocked.stats[[i]], title = names(blocked.stats)[i]) + 
    theme(legend.position = "none")
}

legend <- get_legend(p[[1]] + theme(legend.position = "top", legend.justification = "left"))

fig(width = 16, height = 9)
plot_grid(title, legend, plot_grid(plotlist = p), ncol = 1, rel_heights = c(0.075, 0.075, 0.85))
reset.fig()

# Highlight HVGs
p <-  vector("list", n)
for(i in 1:n) {
    p[[i]] <- VariableFeaturePlot(var = blocked.stats[[i]], hvg = hvg.out4, title = names(blocked.stats)[i]) + 
    theme(legend.position = "none")
}

legend <- get_legend(p[[1]] + theme(legend.position = "top", legend.justification = "left"))

fig(width = 16, height = 9)
plot_grid(title, legend, plot_grid(plotlist = p), ncol = 1, rel_heights = c(0.075, 0.075, 0.85))
reset.fig()

In [ ]:
# For test purposes - All passed genes (include genes affected by ambient contamination)
set.seed(12345)
var.out5 <- modelGeneVarByPoisson(cdScFiltAnnot[rowData(cdScFiltAnnot)$is_pass,], assay.type = "counts", 
                                  block = cdScFiltSub$Sample, BPPARAM = bpp)
hvg.out5 <- getTopHVGs(var.out5, prop = 0.1, var.field = "bio", var.threshold = 0)

df <- rowData(cdScFiltAnnot)[rownames(rowData(cdScFiltAnnot)) %in% hvg.out5,] %>% 
    as.data.frame() %>% arrange(Symbol)
table(Is_Ambient = df$is_ambient)

print("Genes affected by ambient contamination and identified as HVG:")
rownames(df[df$is_ambient == TRUE,])

In [ ]:
# Plot
title <- ggdraw() + draw_label("Passed genes and include ambient contamination", size = 22, fontface = "bold")
blocked.stats <- var.out5$per.block
n <- length(names(blocked.stats))

p <-  vector("list", n)
for(i in 1:n) {
    p[[i]] <- VariableFeaturePlot(var = blocked.stats[[i]], title = names(blocked.stats)[i]) + 
    theme(legend.position = "none")
}

legend <- get_legend(p[[1]] + theme(legend.position = "top", legend.justification = "left"))

fig(width = 16, height = 9)
plot_grid(title, legend, plot_grid(plotlist = p), ncol = 1, rel_heights = c(0.075, 0.075, 0.85))
reset.fig()

# Highlight HVGs
p <-  vector("list", n)
for(i in 1:n) {
    p[[i]] <- VariableFeaturePlot(var = blocked.stats[[i]], hvg = hvg.out5, title = names(blocked.stats)[i]) + 
    theme(legend.position = "none")
}

legend <- get_legend(p[[1]] + theme(legend.position = "top", legend.justification = "left"))

fig(width = 16, height = 9)
plot_grid(title, legend, plot_grid(plotlist = p), ncol = 1, rel_heights = c(0.075, 0.075, 0.85))
reset.fig()

## Decide which model and HVG to use

In [ ]:
# Select which var.out and hvg.out to save
hvg_model <- var.out2
hvg_genes <- hvg.out2

# Subset model DataFrame with HVGs
hvg <- hvg_model[hvg_genes,]
hvg

In [ ]:
# Add HVG to rowData
rowData(cdScFiltAnnot)$is_hvg <- FALSE
rowData(cdScFiltAnnot)[rownames(cdScFiltAnnot) %in% hvg_genes,]$is_hvg <- TRUE
table(rowData(cdScFiltAnnot)$is_hvg)

In [ ]:
# Plotting the top 15 HVGs
plotExpression(cdScFiltAnnot, head(hvg_genes, 15), point_size = 0.6, theme_size = 20)

In [ ]:
# Save to file
write.table(hvg, file = "HVG.xls", sep = "\t", quote = FALSE, col.names = NA)

## Skewed genes

Identify genes that are expressed by a very high proportion of cells, i.e. skewed genes, and check if any HVGs are also skewed genes.

In [ ]:
is_pass_df <- rowData(cdScFiltAnnot)[rowData(cdScFiltAnnot)$is_pass,]
skewedGenes <- isOutlier(is_pass_df$detected, 
                         nmads = 4, type = "higher", log = FALSE)
is_pass_df$is_skewed <- skewedGenes
cut_off_skewed <- attr(skewedGenes, "thresholds")["higher"]

# Plot
ggplot(as.data.frame(is_pass_df), aes(x = detected)) + geom_density(fill = "black") +
geom_vline(xintercept = cut_off_skewed, colour = "red", linetype = "longdash") +
annotate("text", x = cut_off_skewed, y = Inf, label = round(cut_off_skewed, 2), 
         vjust = 2, hjust = -0.2, size = 5) +
theme_classic(base_size = 20) + scale_fill_manual(values = c_sample_col) +
labs(x = "Expressing cell proportion (%)", title = "Expressing cell proportion", fill = "Sample", 
     subtitle = paste(sum(skewedGenes), "genes(s) to the right of cutoff have too high expressing cell proportion"))

print(paste0("Skewed gene cutoff: ", round(cut_off_skewed, 2), "%"))
print(paste0("Total number of skewed genes: ", sum(skewedGenes)))

In [ ]:
print(paste("Total number of skewed genes:", sum(is_pass_df$is_skewed)))
print(paste("Total number of skewed HVGs:", nrow(is_pass_df[is_pass_df$is_skewed & is_pass_df$is_hvg,])))

In [ ]:
# Print skewed HVG names
#rownames(is_pass_df[is_pass_df$is_skewed & is_pass_df$is_hvg,])

In [ ]:
ggplot(as.data.frame(is_pass_df), aes(x = detected, fill = is_hvg)) +
    geom_histogram(color = "white", size = 0.3, bins = 20) +
    scale_fill_manual("HVG Genes", values = c("red", "blue")) +
    geom_vline(xintercept = cut_off_skewed, colour = "red", linetype = "longdash") +
    annotate("text", x = cut_off_skewed, y = Inf, label = round(cut_off_skewed, 2), 
             vjust = 2, hjust = 1.25, size = 5) +
    facet_wrap(~ is_skewed, ncol = 2, scales = "free_y", 
               labeller = as_labeller(function(x) paste("Is skewed:", x))) + 
    theme_classic(base_size = 20) + theme(legend.position = "top") +
    labs(x = "Expressing cell proportion (%)", y = "Number of genes")

__Final Notes__ There are alternative approaches for determining the HVG, especially those based on Coefficient of Variance. The method used here, the variance of the log-expression values, avoids genes with strong expression in only one or two cells. This ensures that the set of top HVGs is not dominated by genes with (mostly uninteresting) outlier expression patterns.

However, it has been mentioned that fitting the trendline to endogenous genes might not always be a good idea.


In [ ]:
save.image("data.RData")

# 8. Data integration (donor effect correction)<a name='section8' />

Before performing dimensionality reduction, we called `fastMNN()` to correct for batch effects in single-cell expression data using a fast version of the mutual nearest neighbors (MNN) method. In multi-sample datasets, we often observe a strong donor effect. This might be due to differences in cell type composition between donors, but the more likely explanation is that of a technical difference in sample preparation processing or uninteresting genotypic differences. 

We will be saving the *corrected* results in a separate `reducedDim` space, so that we can investigate later if we should defined clusters from corrected values to cells from all samples in the chimera dataset.

**Note:** Gene-based analyses should use the uncorrected data with blocking where possible.

<div class="alert alert-warning">
  <strong>Warning!</strong> Requires the <code>batchelor</code> package. Here we use multiple cores, randomized SVD and annoy algorithm for approximate nearest-neighbor detection to speed up this step.
</div>

In [ ]:
if ( requireNamespace("batchelor", quietly = TRUE) ) {
    set.seed(12345)
    corrected <- batchelor::fastMNN(cdScFiltAnnot, subset.row = hvg_genes, batch = cdScFiltAnnot$Sample, 
                                    BSPARAM = BiocSingular::RandomParam(deferred = TRUE), 
                                    BNPARAM = BiocNeighbors::AnnoyParam(), BPPARAM = bpp)
    corrected

    # Print the percentage of variance lost as a diagnostic measure
    metadata(corrected)$merge.info$lost.var

    # Add to reducedDim
    reducedDim(cdScFiltAnnot, "MNN") <- reducedDim(corrected, "corrected")
} else {
    stop("batchelor is not installed, unable to complete data integration")
}

# Print reducedDim name
reducedDimNames(cdScFiltAnnot)

# 9. Dimensionality reduction using HVG<a name='section9' />

1. Principal components analysis (PCA)
2. t-stochastic neighbor embedding (t-SNE)
3. Uniform manifold approximation and projection (UMAP)

## Principal components analysis

We perform the PCA on the log2CPM expression values using the `runPCA()` function. By default, `runPCA()` will compute the first 50 PCs and store them in the `reducedDims()` of the output `sce` object.

Here, we restricting the PCA to a subset of HVGs selected previously (by using `subset_row`) to reduce both computational work and high-dimensional random noise. In particular, while PCA is robust to random noise, an excess of it may cause the earlier PCs to capture noise instead of biological structure.

Usually, we sepcify `ntop = 500` to select 500 genes with the highest variances to use for dimensionality reduction, however the value of `ntop` is ignored if `subset_row`.

**Note:** We did not use `denoisePCA` here because the "variance decomposition results" required as part of the input parameters (`technical`) has different number of genes. The `modelGeneVar`/`modelGeneVarByPoisson` was performed with only the `is_pass` and not `is_ambient` genes, but the `sce` object contains all the genes.

In [ ]:
# runPCA() stores the PCA attributes in the reducedDims
set.seed(12345)
cdScFiltAnnot <- runPCA(cdScFiltAnnot, ncomponents = 50, subset_row = hvg_genes, BPPARAM = bpp)

In [ ]:
# Print reducedDim name
reducedDimNames(cdScFiltAnnot)

# Print the dimension of the PCA reducedDims
dim(reducedDim(cdScFiltAnnot, "PCA"))

# Print the PCA reducedDims structure
str(reducedDim(cdScFiltAnnot, "PCA"))

The attributes of the PC coordinate matrix contain the following elements:

- varExplained - the actual variance explained by each PC. *Not available if using scater verion 1.16.2*
- percentVar - the percentage of variance explained by each PC. This may not sum to 100 if not all PCs are reported.
- rotation - the rotation matrix containing loadings for all genes used in the analysis and for each PC.

### Choosing the number of PCs

How many of the top PCs should we retain for downstream analyses? The choice of the number of PCs `d` is a decision that is analogous to the choice of the number of HVGs to use. Using more PCs will retain more biological signal at the cost of including more noise that might mask said signal.

Nonetheless, we will use some data-driven strategies to guide a suitable choice of `d`. These automated choices are best treated as guidelines as they make some strong assumptions about what variation is “interesting”.

In [ ]:
chosen.elbow <- NULL
chosen.denoised <- NULL
chosen.clustered <- NULL

percent.var <- attr(reducedDim(cdScFiltAnnot, "PCA"), "percentVar")

# 1. Using the elbow point (requires PCAtools)
if ( requireNamespace("PCAtools", quietly = TRUE) ) {
    chosen.elbow <- PCAtools::findElbowPoint(percent.var)
    print(paste("elbow method:", chosen.elbow))
} else {
    print("PCAtools is not installed, skip findElbowPoint()")
}

# 2. Using the technical noise
# Input sce and modelGeneVar stats need to be the same length and order, therefore using cdScFiltSub 
set.seed(12345)
denoised <- denoisePCA(cdScFiltSub, technical = hvg_model, subset.row = hvg_genes, BPPARAM = bpp)
chosen.denoised <- ncol(reducedDim(denoised))
print(paste("technical noise method:", chosen.denoised))

# 3. Based on population structure
clusteredpcs <- getClusteredPCs(reducedDim(cdScFiltAnnot, "PCA"), BPPARAM = bpp)
chosen.clustered <- metadata(clusteredpcs)$chosen
print(paste("population structure method:", chosen.clustered))

In [ ]:
chosen.pc <- data.frame(labels = c("technical noise", "population structure"),
                        values = c(chosen.denoised, chosen.clustered))

if(!is.null(chosen.elbow)) {
        chosen.pc <- rbind(chosen.pc, data.frame(labels = "elbow point", values = chosen.elbow))
}

as.data.frame(percent.var) %>% rownames_to_column() %>%
        mutate(rowname = as.numeric(rowname)) %>%
        ggplot(aes(x = rowname, y = percent.var)) + geom_point(size = 3) +
        geom_vline(mapping = aes(xintercept = values, colour = labels), size = 1, data = chosen.pc) +
        scale_x_continuous(breaks = seq(0, 50, by = 5)) + theme_classic(base_size = 20) + 
        guides(color = guide_legend(title = "Method")) +
        theme(legend.position = "top") + labs(x = "PC", y = "Variance explained (%)")

<div class="alert alert-info">
  <strong>About this dataset: </strong> (edits required)
  <ul>
    <li>The top 15 PCs retain a lot of information, while other PCs contain pregressivelly less.</li>
    <li>However, it is still advisable to use more PCs since they might contain information about rare cell types (in some datasets such as platelets and dendritic cells).</li>
  </ul>
</div>

In [ ]:
round(percent.var[1:20], 2)

## Visualising with PCA

### Top 5 PCs

In [ ]:
p <- plotReducedDim(cdScFiltAnnot, dimred = "PCA", ncomponents = 5, colour_by = "Sample", 
                    point_size = 0.5, point_alpha = 0.1, theme_size = 20) +
    scale_color_manual(values = c_sample_col) +
    guides(color = guide_legend(title = "Sample", override.aes = list(size = 4, alpha = 1)))

fig(width = 12, height = 10)
oldScaterPlotFix(p)
reset.fig()

### PC1 and PC2

In [ ]:
p <- plotReducedDim(cdScFiltAnnot, dimred = "PCA", ncomponents = c(1,2), colour_by = "Sample", 
                    point_size = 2, point_alpha = 0.3, theme_size = 20) + 
    scale_color_manual(values = c_sample_col) +
    guides(color = guide_legend(title = "Sample", override.aes = list(size = 4, alpha = 1)))

oldScaterPlotFix(p)

### PC3 and PC4

In [ ]:
p <- plotReducedDim(cdScFiltAnnot, dimred = "PCA", ncomponents = c(3,4), colour_by = "Sample", 
                    point_size = 2, point_alpha = 0.3, theme_size = 20) +
    scale_color_manual(values = c_sample_col) +
    guides(color = guide_legend(title = "Sample", override.aes = list(size = 4, alpha = 1)))

oldScaterPlotFix(p)

## Visualising with t-SNE

The t-stochastic neighbor embedding (t-SNE) method attempts to find a low-dimensional representation of the data that preserves the distances between each point and its neighbors in the high-dimensional space. Unlike PCA, it is not restricted to linear transformations, nor is it obliged to accurately represent distances between distant populations. This means that it has much more freedom in how it arranges cells in low-dimensional space, enabling it to separate many distinct clusters in a complex population. [OSCA reference](http://bioconductor.org/books/release/OSCA/dimensionality-reduction.html#t-stochastic-neighbor-embedding)

#### n_dimred

By default, all dimensions are used to compute the second set of reduced dimensions. If `n_dimred` is also specified, only the first `n_dimred` columns are used. Alternatively, `n_dimred` can be an integer vector specifying the column indices of the dimensions to use.

#### perplexity

The value of the `perplexity` parameter can have a large effect on the results. By default, the function will set a “reasonable” perplexity that scales with the number of cells in x. (Specifically, it is the number of cells divided by 5, capped at a maximum of 50.) However, it is often worthwhile to manually try multiple values to ensure that the conclusions are robust.

<div class="alert alert-warning">
  <strong>Warning!</strong> When <code>dimred</code> is specified, the PCA step of the <code>Rtsne</code> function is automatically turned off by default, and existing dimensionality reduction results stored in the specified <code>reducedDimNames</code> is used.
</div>

### Use default perplexity

In [ ]:
# Calculate default perplexity to show in plot title
perplexity <- min(50, floor(ncol(cdScFiltAnnot) / 5))
dim_name1 <- "TSNE" # default name
dim_name2 <- "MNN-TSNE"

# Use the scree plot above to select this value, default is 50, i.e. all dimensions
n_dimred <- ncol(reducedDim(cdScFiltAnnot, "PCA"))

# runTSNE() stores the t-SNE coordinates in the reducedDims
set.seed(12345)
cdScFiltAnnot <- runTSNE(cdScFiltAnnot, dimred = "PCA", n_dimred = n_dimred, perplexity = perplexity, 
                         name = dim_name1, n_threads = nthreads, BPPARAM = bpp)

set.seed(12345)
cdScFiltAnnot <- runTSNE(cdScFiltAnnot, dimred = "MNN", perplexity = perplexity, 
                         name = dim_name2, n_threads = nthreads, BPPARAM = bpp)

In [ ]:
p1 <- plotReducedDim(cdScFiltAnnot, dim_name1, colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + ggtitle("Without correction")

p2 <- plotReducedDim(cdScFiltAnnot, dim_name2, colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + ggtitle("After MNN correction")

prow <- plot_grid(
    oldScaterPlotFix(p1) + theme(legend.position = "none"),
    oldScaterPlotFix(p2) + theme(legend.position = "none"),
    align = 'vh', hjust = -1, nrow = 1
)

legend <- get_legend(p1 + 
                     guides(color = guide_legend(title = "Sample", override.aes = list(size = 4, alpha = 1))) +
                     theme(legend.position = "right"))

fig(width = 16, height = 7)
plot_grid(prow, legend, nrow = 1, rel_widths = c(1, 0.15))
reset.fig()

### Try other perplexity values

In [ ]:
# Low
perplexity <- 20
dim_name1 <- paste0("TSNE-", perplexity)
dim_name2 <- paste0("MNN-TSNE-", perplexity)

set.seed(12345)
cdScFiltAnnot <- runTSNE(cdScFiltAnnot, dimred = "PCA", n_dimred = n_dimred, perplexity = perplexity, 
                         name = dim_name1, n_threads = nthreads, BPPARAM = bpp)

set.seed(12345)
cdScFiltAnnot <- runTSNE(cdScFiltAnnot, dimred = "MNN", perplexity = perplexity, 
                         name = dim_name2, n_threads = nthreads, BPPARAM = bpp)

In [ ]:
p1 <- plotReducedDim(cdScFiltAnnot, dimred = dim_name1, colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + ggtitle("Without correction")

p2 <- plotReducedDim(cdScFiltAnnot, dimred = dim_name2, colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + ggtitle("After MNN correction")

prow <- plot_grid(
    oldScaterPlotFix(p1) + theme(legend.position = "none"),
    oldScaterPlotFix(p2) + theme(legend.position = "none"),
    align = 'vh', hjust = -1, nrow = 1
)

legend <- get_legend(p1 + 
                     guides(color = guide_legend(title = "Sample", override.aes = list(size = 4, alpha = 1))) +
                     theme(legend.position = "right"))

fig(width = 16, height = 7)
plot_grid(prow, legend, nrow = 1, rel_widths = c(1, 0.15))
reset.fig()

In [ ]:
# High
perplexity <- 80
dim_name1 <- paste0("TSNE-", perplexity)
dim_name2 <- paste0("MNN-TSNE-", perplexity)

set.seed(12345)
cdScFiltAnnot <- runTSNE(cdScFiltAnnot, dimred = "PCA", n_dimred = n_dimred, perplexity = perplexity, 
                         name = dim_name1, n_threads = nthreads, BPPARAM = bpp)

set.seed(12345)
cdScFiltAnnot <- runTSNE(cdScFiltAnnot, dimred = "MNN", perplexity = perplexity, 
                         name = dim_name2, n_threads = nthreads, BPPARAM = bpp)

In [ ]:
p1 <- plotReducedDim(cdScFiltAnnot, dimred = dim_name1, colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + ggtitle("Without correction")

p2 <- plotReducedDim(cdScFiltAnnot, dimred = dim_name2, colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + ggtitle("After MNN correction")

prow <- plot_grid(
    oldScaterPlotFix(p1) + theme(legend.position = "none"),
    oldScaterPlotFix(p2) + theme(legend.position = "none"),
    align = 'vh', hjust = -1, nrow = 1
)

legend <- get_legend(p1 + 
                     guides(color = guide_legend(title = "Sample", override.aes = list(size = 4, alpha = 1))) +
                     theme(legend.position = "right"))

fig(width = 16, height = 7)
plot_grid(prow, legend, nrow = 1, rel_widths = c(1, 0.15))
reset.fig()

## Visualising with UMAP (`uwot::umap`)

The uniform manifold approximation and projection (UMAP) method is an alternative to t-SNE for non-linear dimensionality reduction. It is roughly similar to t-SNE in that it also tries to find a low-dimensional representation that preserves relationships between neighbors in high-dimensional space. However, the two methods are based on different theory, represented by differences in the various graph weighting equations. [OSCA reference](http://bioconductor.org/books/release/OSCA/dimensionality-reduction.html#t-stochastic-neighbor-embedding)

Compared to t-SNE, the UMAP visualisation tends to have more compact visual clusters with more empty space between them. It also attempts to preserve more of the global structure than t-SNE.

UMAP has its own suite of hyperparameters that affect the visualisation. Of these, the number of neighbors (`n_neighbors`) and the minimum distance between embedded points (`min_dist`) have the greatest effect on the granularity of the output. If these values are too low, random noise will be incorrectly treated as high-resolution structure, while values that are too high will discard fine structure altogether in favor of obtaining an accurate overview of the entire dataset. Again, it is a good idea to test a range of values for these parameters to ensure that they do not compromise any conclusions drawn from a UMAP plot.

#### n_dimred

By default, all dimensions are used to compute the second set of reduced dimensions. If `n_dimred` is also specified, only the first `n_dimred` columns are used. Alternatively, `n_dimred` can be an integer vector specifying the column indices of the dimensions to use.

#### n_neighbors

This sets the number of nearest neighbors to identify when constructing the initial graph. Larger values result in more global views of the manifold, while smaller values result in more local data being preserved. In general values should be in the range 2 to 100. The default value set by `uwot` is 15 (*seurat uses 30*).

#### spread

The effective scale of embedded points. In combination with `min_dist`, this determines how clustered/clumped the embedded points are. The default value set by `uwot` is 1 (*seurat also uses 1*).

#### min_dist

The effective minimum distance between embedded points. Smaller values will result in a more clustered/clumped embedding where nearby points on the manifold are drawn closer together, while larger values will result on a more even dispersal of points. The value should be set relative to the `spread` value, which determines the scale at which embedded points will be spread out. The default value set by `uwot` is 0.01 (*seurat also uses 0.3*).

<div class="alert alert-warning">
  <strong>Warning!</strong> When <code>dimred</code> is specified, no additional feature selection or standardization is performed, and existing dimensionality reduction results stored in the specified <code>reducedDimNames</code> is used.
</div>

### Use default uwot settings (very compact)

In [ ]:
# Use the scree plot above to select this value, default is 50, i.e. all dimensions
n_dimred <- ncol(reducedDim(cdScFiltAnnot, "PCA"))
dim_name1 <- "UMAP" # default name
dim_name2 <- "MNN-UMAP"

set.seed(12345)
cdScFiltAnnot <- runUMAP(cdScFiltAnnot, dimred = "PCA", n_dimred = n_dimred, 
                         n_neighbors = 15, spread = 1, min_dist = 0.01, 
                         name = dim_name1, n_threads = nthreads, BPPARAM = bpp)

set.seed(12345)
cdScFiltAnnot <- runUMAP(cdScFiltAnnot, dimred = "MNN", 
                         n_neighbors = 15, spread = 1, min_dist = 0.01, 
                         name = dim_name2, n_threads = nthreads, BPPARAM = bpp)

In [ ]:
p1 <- plotReducedDim(cdScFiltAnnot, dimred = dim_name1, colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + ggtitle("Without correction")

p2 <- plotReducedDim(cdScFiltAnnot, dimred = dim_name2, colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + ggtitle("After MNN correction")

prow <- plot_grid(
    oldScaterPlotFix(p1) + theme(legend.position = "none"),
    oldScaterPlotFix(p2) + theme(legend.position = "none"),
    align = 'vh', hjust = -1, nrow = 1
)

legend <- get_legend(p1 + 
                     guides(color = guide_legend(title = "Sample", override.aes = list(size = 4, alpha = 1))) +
                     theme(legend.position = "right"))

fig(width = 16, height = 7)
plot_grid(prow, legend, nrow = 1, rel_widths = c(1, 0.15))
reset.fig()

### Trying other settings

In [ ]:
dim_name1 <- "UMAP-Large"
dim_name2 <- "MNN-UMAP-Large"

set.seed(12345)
cdScFiltAnnot <- runUMAP(cdScFiltAnnot, dimred = "PCA", n_dimred = n_dimred, 
                         n_neighbors = 30, spread = 1, min_dist = 0.3, # Default settings in seurat
                         name = dim_name1, n_threads = nthreads, BPPARAM = bpp)

set.seed(12345)
cdScFiltAnnot <- runUMAP(cdScFiltAnnot, dimred = "MNN",
                         n_neighbors = 30, spread = 1, min_dist = 0.3, # Default settings in seurat
                         name = dim_name2, n_threads = nthreads, BPPARAM = bpp)

In [ ]:
p1 <- plotReducedDim(cdScFiltAnnot, dimred = dim_name1, colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + ggtitle("Without correction")

p2 <- plotReducedDim(cdScFiltAnnot, dimred = dim_name2, colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + ggtitle("After MNN correction")

prow <- plot_grid(
    oldScaterPlotFix(p1) + theme(legend.position = "none"),
    oldScaterPlotFix(p2) + theme(legend.position = "none"),
    align = 'vh', hjust = -1, nrow = 1
)

legend <- get_legend(p1 + 
                     guides(color = guide_legend(title = "Sample", override.aes = list(size = 4, alpha = 1))) +
                     theme(legend.position = "right"))

fig(width = 16, height = 7)
plot_grid(prow, legend, nrow = 1, rel_widths = c(1, 0.15))
reset.fig()

In [ ]:
dim_name1 <- "UMAP-Larger"
dim_name2 <- "MNN-UMAP-Larger"

set.seed(12345)
cdScFiltAnnot <- runUMAP(cdScFiltAnnot, dimred = "PCA", n_dimred = n_dimred, 
                         n_neighbors = 50, spread = 1, min_dist = 0.5, # Ever larger n_neighbors & min_dist
                         name = dim_name1, n_threads = nthreads, BPPARAM = bpp)

set.seed(12345)
cdScFiltAnnot <- runUMAP(cdScFiltAnnot, dimred = "MNN",
                         n_neighbors = 50, spread = 1, min_dist = 0.5, # Ever larger n_neighbors & min_dist
                         name = dim_name2, n_threads = nthreads, BPPARAM = bpp)

In [ ]:
p1 <- plotReducedDim(cdScFiltAnnot, dimred = dim_name1, colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + ggtitle("Without correction")

p2 <- plotReducedDim(cdScFiltAnnot, dimred = dim_name2, colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + ggtitle("After MNN correction")

prow <- plot_grid(
    oldScaterPlotFix(p1) + theme(legend.position = "none"),
    oldScaterPlotFix(p2) + theme(legend.position = "none"),
    align = 'vh', hjust = -1, nrow = 1
)

legend <- get_legend(p1 + 
                     guides(color = guide_legend(title = "Sample", override.aes = list(size = 4, alpha = 1))) +
                     theme(legend.position = "right"))

fig(width = 16, height = 7)
plot_grid(prow, legend, nrow = 1, rel_widths = c(1, 0.15))
reset.fig()

## Diagnosing batch effects

Examining the above dimensionality reduction plots, are there batch effect in this dataset? [Extended Reading](http://bioconductor.org/books/release/OSCA/integrating-datasets.html#using-corrected-values)

<div class="alert alert-info">
  <strong>About this dataset: </strong> (edits required)
  <ul>
    <li>We see a some separation between cells from different samples in the uncorrected data.</li>
  </ul>
</div>

In [ ]:
# Print reducedDim name
reducedDimNames(cdScFiltAnnot)

In [ ]:
save.image("data.RData")

# 10. Clustering of cells<a name='section10' />

The goal is to split the cells in the dataset into clusters, such that:

1. the cellular expression in the same cluster are as similar as possible,
2. the cellular expression in different clusters are highly distinct

There are numerous methods of clustering; hierarchical clustering (+`dynamicTreeCut`), k-means, k-medoids, GMM, GP-LVM and so on. Here the dynamic-cut-tree will be used.

<div class="alert alert-danger">
  <strong>Danger!</strong> Hierarchical clustering is not a scalable approach with respect to the number of cells.
</div>

## Use hierarchical clustering

Hierarchical clustering is an ancient technique that aims to generate a dendrogram containing a hierarchy of samples. This is most commonly done by greedily agglomerating samples into clusters, then agglomerating those clusters into larger clusters, and so on until all samples belong to a single cluster.

### About Dynamic Tree Cut

Hierarchical clustering is a widely used method for detecting clusters in genomic data. Clusters are defined by cutting branches off the dendrogram. A common but inflexible method uses a constant height cutoff value; this method exhibits suboptimal performance on complicated dendrograms. The **Dynamic Tree Cut** R library implements novel dynamic branch cutting methods for detecting clusters in a dendrogram depending on their shape. Compared to the constant height cutoff method, these techniques offer the following advantages: 

1. they are capable of identifying nested clusters
2. they are flexible --- cluster shape parameters can be tuned to suit the application at hand
3. they are suitable for automation
4. they can optionally combine the advantages of hierarchical clustering and partitioning around medoids, giving better detection of outliers.

### Clustering cells into putative subpopulations

Hierarchical clustering is performed on the Euclidean distances between cells, using Ward’s criterion to minimize the total variance within each cluster. This yields a dendrogram that groups together cells with similar expression patterns across the chosen genes. 

Clusters are explicitly defined by applying a dynamic tree cut [Langfelder et al., 2008](https://academic.oup.com/bioinformatics/article/24/5/719/200751) to the dendrogram. This exploits the shape of the branches in the dendrogram to refine the cluster definitions, and is more appropriate than cutree for complex dendrograms. Greater control of the empirical clusters can be obtained by manually specifying cutHeight in `cutreeDynamic`.

An alternative approach is to cluster on a matrix of distances derived from correlations (e.g., as in `quickCluster`). This is more robust to noise and normalisation errors, but is also less sensitive to subtle changes in the expression profiles.

<div class="alert alert-warning">
    <strong>Warning!</strong> Clustering using <code>logcounts</code> is now removed as the silhouette width is always much lower, which suggest weak classification.
</div>

In [ ]:
# Stores hclust + cut.dynamic labels
hclust.clusters <- vector(mode = "list")
my.dist <- vector(mode = "list")

my.dist[["PCA"]] <- dist(reducedDim(cdScFiltAnnot, "PCA"), method = "euclidean")
my.dist[["MNN"]] <- dist(reducedDim(cdScFiltAnnot, "MNN"), method = "euclidean")

if ( requireNamespace("bluster", quietly = TRUE) ) {
    hclust.clusters[["PCA"]] <- bluster::clusterRows(reducedDim(cdScFiltAnnot, "PCA"), 
                                        bluster::HclustParam(method = "ward.D2", cut.dynamic = TRUE))
    hclust.clusters[["MNN"]] <- bluster::clusterRows(reducedDim(cdScFiltAnnot, "MNN"), 
                                        bluster::HclustParam(method = "ward.D2", cut.dynamic = TRUE))
} else {
    hclust.clusters[["PCA"]] <- cutreeDynamic(hclust(my.dist[["PCA"]], method = "ward.D2"), 
                                              distM = as.matrix(my.dist[["PCA"]]))
    hclust.clusters[["MNN"]] <- cutreeDynamic(hclust(my.dist[["MNN"]], method = "ward.D2"), 
                                              distM = as.matrix(my.dist[["MNN"]]))
}

In [ ]:
print("Cluster assignments:")
for(i in 1:length(hclust.clusters)) {
    tab <- table(colData(cdScFiltAnnot)$Sample, hclust.clusters[[i]])
    names(dimnames(tab))[2] <- names(hclust.clusters)[i]
    print(tab)
}

### Assessing cluster separation

We check the separation of the clusters using the silhouette width. Each cluster would ideally contain large positive silhouette widths, indicating that it is well-separated from other clusters. 

#### Silhouette cluster validation plot

The silhouette plot displays a measure of how close each point in one cluster is to points in the neighboring clusters.

Silhouette width can be interpreted as follows:

- Cells with a large Si (almost 1) are very well clustered.
- A small Si (around 0) means that the observation lies between two clusters.
- Cells with a negative Si are probably placed in the wrong cluster. (so cluster is less stable and trustworthy)


#### Background to cluster validation

#### Internal measures for cluster validation

In this section, we describe the most widely used clustering validation indices. Recall that the goal of partitioning clustering algorithms (Part @ref(partitioning-clustering)) is to split the data set into clusters of objects, such that:

- the objects in the same cluster are as similar as possible,
- and the objects in different clusters are highly distinct
- [Choosing the Best Clustering Algorithms](https://www.datanovia.com/en/lessons/choosing-the-best-clustering-algorithms/), [Cluster Validation Statistics: Must Know Methods](https://www.datanovia.com/en/lessons/cluster-validation-statistics-must-know-methods/)

**We want the average distance within cluster to be as small as possible; and the average distance between clusters to be as large as possible.**

Internal validation measures often reflect the compactness, the connectedness and the separation of the cluster partitions.

1. Compactness or cluster cohesion: Measures how close are the objects within the same cluster. A lower within-cluster variation is an indicator of a good compactness (i.e., a good clustering). The different indices for evaluating the compactness of clusters are base on distance measures such as the cluster-wise within average/median distances between observations.
2. Separation: Measures how well-separated a cluster is from other clusters. The indices used as separation measures include:
    * distances between cluster centers
    * the pairwise minimum distances between objects in different clusters
3. Connectivity: corresponds to what extent items are placed in the same cluster as their nearest neighbors in the data space. The connectivity has a value between 0 and infinity and should be minimized.

Generally most of the indices used for internal clustering validation combine compactness and separation measures as follow:

$Index = \frac{\alpha * Seperation}{\beta * Compactness}$
where $\alpha$ and $\beta$ are weights.

#### Silhouette coefficient

The silhouette analysis measures how well an observation is clustered and it estimates the average distance between clusters. The silhouette plot displays a measure of how close each point in one cluster is to points in the neighboring clusters.

For each observation $i$, the silhouette width $s_i$ is calculated as follows:

1. For each observation $i$, calculate the average dissimalirty $\alpha_i$ between $i$ and all other points of the cluster which $i$ belongs.
2. For all other clusters $C$, to which $i$ does not belong, calculate the average dissimilarity $d(i,C)$ of $i$ to all observations of $C$. The smallest of these $d(i,C)$ is defined as $b_i = min_C d(i,C)$. The value of $b_i$ can be seen as the dissimilarity between i and its __neighbor__ cluster, i.e. the nearest one to which it does not belong.
3. Finally the silhouette width of the observation $i$ is defined by the formula: $S_i = \frac{(b_i - a_i)}{max(a_i,b_i)}$

Silhouette width can be interpreted as follow:

- Observations with a large Si (almost 1) are very well clustered.
- A small Si (around 0) means that the observation lies between two clusters.
- Observations with a negative Si are probably placed in the wrong cluster.

In [ ]:
# Use bluster::approxSilhouette
plotSilhouette <- function(reducedDim, clusters, printDiff = TRUE, plot = TRUE) {
    if ( requireNamespace("bluster", quietly = TRUE) ) {
        # Performing the calculations on the PC coordinates, like before.
        sil.approx <- bluster::approxSilhouette(reducedDim, clusters = clusters)
        sil.data <- as.data.frame(sil.approx)
        sil.data$closest <- ifelse(sil.data$width > 0, clusters, sil.data$other)
        sil.data$cluster <- factor(clusters)
        sil.data$isSame <- sil.data$closest == sil.data$cluster
        
        print("Silhouette width summary:")
        print(summary(sil.data$width))
        
        if(printDiff) {
            print(sil.data %>% group_by(cluster, isSame) %>% summarize(N = n()) %>% 
                  mutate("% Different" = round(N / sum(N) * 100, 2)) %>% ungroup() %>% 
                  complete(cluster, isSame, fill = list(N = 0, "% Different" = 0)) %>%
                  dplyr::filter(isSame == FALSE) %>% dplyr::select(-c(N, isSame)), n = 30)
        }

        if(plot) {
            p <- ggplot(sil.data, aes(x = cluster, y = width, colour = factor(closest))) + 
                theme_classic(base_size = 20) + geom_quasirandom(size = 1, alpha = 0.8, method = "smiley") +
                geom_hline(yintercept = mean(sil.data$width), size = 1, linetype = "dashed", color = "red" ) +
                geom_hline(yintercept = 0, size = 0.5, color = "black" ) + ylab("Silhouette width Si")
        
            ncol = ceiling(length(table(clusters))/25) # 25 in a column
            p <- p + guides(color = guide_legend(title = "Cluster", ncol = ncol, 
                                                 override.aes = list(size = 4, alpha = 1)))
        
            if(length(table(clusters)) <= 30) {
                p <- p + scale_color_manual(values = c_clust_col) # use c_clust_col 
            }
            print(p)
        }
    } else {
        print("bluster is not installed, skip visualising approximate silhouette width.")
    }
}

In [ ]:
fig(width = 16, height = 7)
plotSilhouette(reducedDim(cdScFiltAnnot, "PCA"), hclust.clusters[["PCA"]])
reset.fig()

In [ ]:
fig(width = 16, height = 7)
plotSilhouette(reducedDim(cdScFiltAnnot, "MNN"), hclust.clusters[["MNN"]])
reset.fig()

<div class="alert alert-info">
  <strong>About this dataset: </strong> (edits required)
  <ul>
    <li>It looks like the clusters are not very stable.</li>
  </ul>
</div>

### Show assigned clusters

We add the cluster assignments back into the `sce` object as a factor in the column metadata (`label`). This allows us to visualise the distribution of clusters in a t-SNE plot.

In [ ]:
# Plots colored by samples
p1 <- vector('list', 2)

p1[[1]] <- plotReducedDim(cdScFiltAnnot, "TSNE", colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + theme(legend.position = "none") + 
    ggtitle("Without correction", subtitle = "Coloured by Sample")
p1[[1]] <- oldScaterPlotFix(p1[[1]])

p1[[2]] <- plotReducedDim(cdScFiltAnnot, "MNN-TSNE", colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + theme(legend.position = "none") + 
    ggtitle("After MNN correction", subtitle = "Coloured by Sample")
p1[[2]] <- oldScaterPlotFix(p1[[2]])

# Plots colored by clusters
p2 <- vector('list', length(hclust.clusters))

is.max = 0
for(i in 1:length(hclust.clusters)) {
    colLabels(cdScFiltAnnot) <- factor(hclust.clusters[[i]])
    
    # Choose assignment with maximum number of clusters
    if(length(levels(colLabels(cdScFiltAnnot))) > is.max) {
        is.max <- length(levels(colLabels(cdScFiltAnnot)))
        is.max <- setNames(is.max, i)
    }

    if(names(hclust.clusters)[i] == "MNN") {
        p2[[i]] <- plotReducedDim(cdScFiltAnnot, "MNN-TSNE", colour_by = "label", text_by = "label", 
                     point_size = 1, point_alpha = 0.2, text_size = 10, text_colour = "black", theme_size = 20) + 
        scale_color_manual(values = c_clust_col) + theme(legend.position = "none") +
        ggtitle("After MNN correction", subtitle = "MNN")

    } else {
        p2[[i]] <- plotReducedDim(cdScFiltAnnot, "TSNE", colour_by = "label", text_by = "label", 
                     point_size = 1, point_alpha = 0.2, text_size = 10, text_colour = "black", theme_size = 20) +
        scale_color_manual(values = c_clust_col) + theme(legend.position = "none") +
        ggtitle("Without correction", subtitle = names(hclust.clusters)[i])
    }
    p2[[i]] <- oldScaterPlotFix(p2[[i]])
}

In [ ]:
# Join plots
prow1 <- plot_grid(plotlist = p1, align = 'vh', hjust = -1, nrow = 1)
prow2 <- plot_grid(plotlist = p2, align = 'vh', hjust = -1, nrow = 1)

legend1 <- get_legend(p1[[1]] + 
                      guides(color = guide_legend(title = "Sample", override.aes = list(size = 4, alpha = 1))) + 
                      theme(legend.position = "right"))

# Show legend from plot with maximum number of clusters
legend2 <- get_legend(p2[[as.numeric(names(is.max))]] + 
                      guides(color = guide_legend(title = "Hierarchical\nclustering",
                                                  ncol = 1, # ncol can be 1 is number of clusters is < 17
                                                  override.aes = list(size = 4, alpha = 1))) +
                      theme(legend.position = "right"))

fig(width = 16, height = 12)
plot_grid(
    plot_grid(prow1, legend1, nrow = 1, rel_widths = c(0.8, 0.2)),
    plot_grid(prow2, legend2, nrow = 1, rel_widths = c(0.8, 0.2)),
    ncol = 1, rel_heights = c(0.5, 0.5)
)
reset.fig()

<div class="alert alert-info">
  <strong>About this dataset: </strong> (edits required)
  <ul>
    <li>We see an unbalanced contributions of cells from different samples to each cluster in the uncorrected data.</li>
  </ul>
</div>

## Use graph-based clustering

Popularized by its use in Seurat, graph-based clustering is a flexible and scalable technique for clustering large scRNA-seq datasets. The major advantage of graph-based clustering lies in its scalability. It only requires a  
k-nearest neighbor search that can be done in log-linear time on average. Graph construction avoids making strong assumptions about the shape of the clusters or the distribution of cells within each cluster.

From a practical perspective, each cell is forcibly connected to a minimum number of neighboring cells, which reduces the risk of generating many uninformative clusters consisting of one or two outlier cells. The main drawback of graph-based methods is that, after graph construction, no information is retained about relationships beyond the neighboring cells. [OSCA reference](http://bioconductor.org/books/release/OSCA/clustering.html#clustering-graph)

#### About `k`

Set `k` to specify the number of nearest neighbors to consider during graph construction. The choice of `k` controls the connectivity of the graph and the resolution of community detection algorithms. Smaller values of `k` will generally yield smaller, finer clusters, while increasing `k` will increase the connectivity of the graph and make it more difficult to resolve different communities. The value of `k` can be roughly interpreted as the anticipated size of the smallest subpopulation. 

#### Different methods for finding communities in `igraph`

In this notebook, we use the Walktrap algorithm (`cluster_walktrap`) to detect community structure. This is also the method demonstrated in [OSCA](http://bioconductor.org/books/release/OSCA/clustering.html#implementation). There are other methods in `igraph` ([doc](https://igraph.org/r/doc/communities.html)): 

- cluster_edge_betweenness
- cluster_fast_greedy
- cluster_label_prop
- cluster_leading_eigen
- cluster_louvain
- cluster_optimal
- cluster_spinglass
- cluster_walktrap

#### Specifying the number of clusters

The `igraph`'s Walktrap community finding algorithm is a hierarchical algorithm. This implies that there is an underlying hierarchy in the communities object, and we can use the `cut_at` function to cut the merge tree at the desired place and returns a membership vector. The desired place can be expressed as the desired number of communities (e.g. `no=10`) or as the number of merge steps to make (e.g. `steps = 5`). The function gives an error message if called with a non-hierarchical method.

```
g <- buildSNNGraph(sce, k = 10, use.dimred = "PCA")
clu <- igraph::cut_at(cluster_walktrap(g), no = 10) # returns 10 clusters
```

In [ ]:
# Stores graph-based walktrap labels
# Use multiple cores and annoy algorithm for approximate nearest-neighbor detection to speed up this step
graph.clusters <- vector(mode = "list")
communities <- vector(mode = "list")

k <- 50

if ( requireNamespace("bluster", quietly = TRUE) ) {
    communities[["PCA"]] <- bluster::clusterRows(reducedDim(cdScFiltAnnot, "PCA"), 
                                                 bluster::NNGraphParam(cluster.fun = "walktrap", k = k, 
                                                                       BNPARAM = BiocNeighbors::AnnoyParam(), 
                                                                       BPPARAM = bpp), full = TRUE)
    graph.clusters[["PCA"]] <- communities[["PCA"]]$clusters
    
    communities[["MNN"]] <- bluster::clusterRows(reducedDim(cdScFiltAnnot, "MNN"), 
                                                 bluster::NNGraphParam(cluster.fun = "walktrap", k = k, 
                                                                       BNPARAM = BiocNeighbors::AnnoyParam(), 
                                                                       BPPARAM = bpp), full = TRUE)
    graph.clusters[["MNN"]] <- communities[["MNN"]]$cluster
} else {
    communities[["PCA"]] <- cluster_walktrap(buildSNNGraph(cdScFiltAnnot, k = k, use.dimred = "PCA", 
                                                           BNPARAM = BiocNeighbors::AnnoyParam(), BPPARAM = bpp))
    graph.clusters[["PCA"]] <- communities[["PCA"]]$membership
    
    communities[["MNN"]] <- cluster_walktrap(buildSNNGraph(cdScFiltAnnot, k = k, use.dimred = "MNN", 
                                                           BNPARAM = BiocNeighbors::AnnoyParam(), BPPARAM = bpp))
    graph.clusters[["MNN"]] <- communities[["MNN"]]$membership
}

In [ ]:
print("Cluster assignments:")
for(i in 1:length(graph.clusters)) {
    tab <- table(colData(cdScFiltAnnot)$Sample, graph.clusters[[i]])
    names(dimnames(tab))[2] <- names(graph.clusters)[i]
    print(tab)
    
    plotSilhouette(reducedDim(cdScFiltAnnot, names(graph.clusters)[i]), graph.clusters[[i]], 
                   printDiff = FALSE, plot = FALSE)
}

### Manually tuning clustering resolution

You can increase `k` to obtain less resolved clusters or descrease to obtain more resolved clusters. Here we are using `cut_at` to set the number of clusters.

In [ ]:
# Use cut_at to set cluster size
n <- 18

if ( requireNamespace("bluster", quietly = TRUE) ) {
    graph.clusters[["PCA"]] <- cut_at(communities[["PCA"]]$objects$communities, n = n)
    graph.clusters[["MNN"]] <- cut_at(communities[["MNN"]]$objects$communities, n = n)
} else {
    graph.clusters[["PCA"]] <- cut_at(communities[["PCA"]], n = n)  
    graph.clusters[["MNN"]] <- cut_at(communities[["MNN"]], n = n)
}

print("Cluster assignments:")
for(i in 1:length(graph.clusters)) {
    tab <- table(colData(cdScFiltAnnot)$Sample, graph.clusters[[i]])
    names(dimnames(tab))[2] <- names(graph.clusters)[i]
    print(tab)
    
    plotSilhouette(reducedDim(cdScFiltAnnot, names(graph.clusters)[i]), graph.clusters[[i]], 
                   printDiff = FALSE, plot = FALSE)
}

In [ ]:
fig(width = 16, height = 7)
plotSilhouette(reducedDim(cdScFiltAnnot, "PCA"), graph.clusters[["PCA"]])
reset.fig()

In [ ]:
fig(width = 16, height = 7)
plotSilhouette(reducedDim(cdScFiltAnnot, "MNN"), graph.clusters[["MNN"]])
reset.fig()

<div class="alert alert-info">
  <strong>About this dataset: </strong> (edits required)
  <ul>
    <li><code>cut_at</code> with 18 clusters yields relatively higher silhouette width.</li>
    <li>It looks like the clusters are stable with the exception of cluster 1, 3 and 7.</li>
  </ul>
</div>

### Show assigned clusters

We add the cluster assignments back into the `sce` object as a factor in the column metadata (`label`). This allows us to visualise the distribution of clusters in a t-SNE plot.

In [ ]:
# Plots colored by samples
p1 <- vector('list', 2)

p1[[1]] <- plotReducedDim(cdScFiltAnnot, "TSNE", colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + theme(legend.position = "none") + 
ggtitle("Without correction", subtitle = "Coloured by Sample")
p1[[1]] <- oldScaterPlotFix(p1[[1]])

p1[[2]] <- plotReducedDim(cdScFiltAnnot, "MNN-TSNE", colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.2, theme_size = 20) +
    scale_color_manual(values = c_sample_col) + theme(legend.position = "none") + 
ggtitle("After MNN correction", subtitle = "Coloured by Sample")
p1[[2]] <- oldScaterPlotFix(p1[[2]])

# Plots colored by clusters
p2 <- vector('list', length(graph.clusters))

is.max = 0
for(i in 1:length(graph.clusters)) {
    colLabels(cdScFiltAnnot) <- factor(graph.clusters[[i]])
    
    # Choose assignment with maximum number of clusters
    if(length(levels(colLabels(cdScFiltAnnot))) > is.max) {
        is.max <- length(levels(colLabels(cdScFiltAnnot)))
        is.max <- setNames(is.max, i)
    }

    if(names(graph.clusters)[i] == "MNN") {
        p2[[i]] <- plotReducedDim(cdScFiltAnnot, "MNN-TSNE", colour_by = "label", text_by = "label", 
                     point_size = 1, point_alpha = 0.2, text_size = 10, text_colour = "black", theme_size = 20) + 
        scale_color_manual(values = c_clust_col) + theme(legend.position = "none") +
        ggtitle("After MNN correction", subtitle = "MNN")
    } else {
        p2[[i]] <- plotReducedDim(cdScFiltAnnot, "TSNE", colour_by = "label", text_by = "label", 
                     point_size = 1, point_alpha = 0.2, text_size = 10, text_colour = "black", theme_size = 20) +
        scale_color_manual(values = c_clust_col) + theme(legend.position = "none") +
        ggtitle("Without correction", subtitle = names(graph.clusters)[i])
    }
    p2[[i]] <- oldScaterPlotFix(p2[[i]])
}

In [ ]:
# Join plots
prow1 <- plot_grid(plotlist = p1, align = 'vh', hjust = -1, nrow = 1)
prow2 <- plot_grid(plotlist = p2, align = 'vh', hjust = -1, nrow = 1)

legend1 <- get_legend(p1[[1]] + 
                      guides(color = guide_legend(title = "Sample", override.aes = list(size = 4, alpha = 1))) + 
                      theme(legend.position = "right"))

# Show legend from plot with maximum number of clusters
legend2 <- get_legend(p2[[as.numeric(names(is.max))]] + 
                      guides(color = guide_legend(title = "Graph-based\nclustering",
                                                  ncol = 2, # ncol can be 1 is number of clusters is < 17
                                                  override.aes = list(size = 4, alpha = 1))) +
                      theme(legend.position = "right"))

fig(width = 16, height = 12)
plot_grid(
    plot_grid(prow1, legend1, nrow = 1, rel_widths = c(0.8, 0.2)),
    plot_grid(prow2, legend2, nrow = 1, rel_widths = c(0.8, 0.2)),
    ncol = 1, rel_heights = c(0.5, 0.5)
)
reset.fig()

<div class="alert alert-info">
  <strong>About this dataset: </strong> (edits required)
  <ul>
    <li>The contributions of cells from different samples to each cluster is quite balanced in both uncorrected and corrected data.</li>
  </ul>
</div>

## Add assigned clusters to `sce` objects

Here we assigned the graph-based clusters to the default "label" column using the `colLabels` function. We also add 2 additional labels (`hclustClusters` and `graphClusters`) so that the BCF R Shiny app can find the assignments by the graph-based and hclust+dynamicTreeCut methods.

<div class="alert alert-warning">
  <strong>Warning!</strong> Based on the diagnostic checks by examing dimensionality reduction plots and clustering results, decide if cluster labels from using corrected values is preferred.
</div>

In [ ]:
# Here we assigned the graph-based clusters to the default "label" column
colLabels(cdScFiltAnnot) <- factor(graph.clusters[["MNN"]])
colLabels(cdScFiltSub) <- factor(graph.clusters[["MNN"]])

# Also save to label for R Shiny app
colData(cdScFiltAnnot)$hclustClusters <- factor(hclust.clusters[["MNN"]])
colData(cdScFiltSub)$hclustClusters <- factor(hclust.clusters[["MNN"]])
colData(cdScFiltAnnot)$graphClusters <- factor(graph.clusters[["MNN"]])
colData(cdScFiltSub)$graphClusters <- factor(graph.clusters[["MNN"]])

In [ ]:
table(hclust = colData(cdScFiltAnnot)$hclustClusters, graph = colData(cdScFiltSub)$graphClusters)

## Heatmap of the expression of the HVGs

### Using cluster assignment from hierarchical clustering

In [ ]:
hvg.exprs <- logcounts(cdScFiltAnnot[rowData(cdScFiltAnnot)$is_hvg,])
heat.vals <- hvg.exprs - rowMeans(hvg.exprs)
heat.vals <- as.matrix(heat.vals)

# Using ComplexHeatmap package
ha <- HeatmapAnnotation(Cluster = colData(cdScFiltAnnot)$hclustClusters, 
                        col = list(Cluster = setNames(c_clust_col[levels(colData(cdScFiltAnnot)$hclustClusters)], 
                                                      levels(colData(cdScFiltAnnot)$hclustClusters))))

ht <- Heatmap(heat.vals, name = "HVG Expression", border = TRUE, top_annotation = ha, 
              cluster_rows = TRUE, show_row_names = TRUE, 
              cluster_columns = TRUE, clustering_method_columns = "ward.D2", show_column_names = FALSE, 
              row_title = paste0(nrow(heat.vals)," HVGs"), row_title_rot = 90, 
              row_title_gp = gpar(fontsize = 14), row_names_gp = gpar(fontsize = 8),
              column_dend_reorder = TRUE, row_dend_reorder = TRUE, 
              heatmap_legend_param = list(title = "Expression", labels_gp = gpar(font = 5), 
                                          title_position = "lefttop-rot"))

fig(width = 14, height = 10)
ht
reset.fig()

In [ ]:
pdf("heatmap_with_HVG_Design.pdf", width = 20, height = 100)
ht
dev.off()

In [ ]:
# Plot heatmap and cluster based on hierarchical clustering
hclust_colors = list(hclustClusters = setNames(c_clust_col[levels(colData(cdScFiltAnnot)$hclustClusters)], 
                                               levels(colData(cdScFiltAnnot)$hclustClusters)))

## Using scater's plotHeatmap function
# The column_annotation_colors is not supported in the current scater version 1.16.2, 
# but will become available in 1.17.X, this will allow us to set cluster colours
fig(width = 14, height = 10)
if(packageVersion("scater") == "1.16.2") {
    plotHeatmap(cdScFiltAnnot, features = hvg_genes, 
                order_columns_by = "hclustClusters", clustering_method = "ward.D2", show_colnames = FALSE)
} else {
    plotHeatmap(cdScFiltAnnot, features = hvg_genes, 
                order_columns_by = "hclustClusters", clustering_method = "ward.D2", 
                column_annotation_colors = hclust_colors, show_colnames = FALSE)
}
reset.fig()

### Using cluster assignment from graph-based clustering

In [ ]:
graph_colors = list(graphClusters = setNames(c_clust_col[levels(cdScFiltAnnot$graphClusters)], 
                                     levels(cdScFiltAnnot$graphClusters)))

fig(width = 14, height = 10)
if(packageVersion("scater") == "1.16.2") {
    plotHeatmap(cdScFiltAnnot, features = hvg_genes, 
                order_columns_by = "graphClusters", clustering_method = "ward.D2", show_colnames = FALSE)
} else {
    plotHeatmap(cdScFiltAnnot, features = hvg_genes, 
                order_columns_by = "graphClusters", clustering_method = "ward.D2", 
                column_annotation_colors = graph_colors, show_colnames = FALSE)
}
reset.fig()

## t-SNE and UMAP plots

<div class="alert alert-warning">
    <strong>Warning!</strong> change the <code>dimred</code> name to view plot in the corresponding reduced dimension results.
</div>

In [ ]:
## t-SNE with cluster allocation
p1 <- plotReducedDim(cdScFiltAnnot, dimred = "MNN-TSNE", colour_by = "graphClusters", text_by = "graphClusters", 
                     point_size = 1, text_size = 10, text_colour = "black", point_alpha = 0.5, theme_size = 20) +
    scale_color_manual(values = c_clust_col) +
    guides(color = guide_legend(title = "Cluster", override.aes = list(size = 4, alpha = 1))) +
    ggtitle("t-SNE coloured by clusters (graph-based)")

p1 <- oldScaterPlotFix(p1)
p1

pdf("tSNE_with_cluster.pdf")
p1
dev.off()

In [ ]:
## t-SNE with cell-cycle
p2 <- plotReducedDim(cdScFiltAnnot, dimred = "MNN-TSNE", colour_by = "CellCycle", 
                     point_size = 1, point_alpha = 0.5, theme_size = 20) +
    scale_color_manual(values = c_phase_col) +
    guides(color = guide_legend(title = "CellCycle", override.aes = list(size = 4, alpha = 1))) +
    ggtitle("t-SNE coloured by cell-cycle")

p2 <- oldScaterPlotFix(p2)
p2

In [ ]:
## t-SNE with sample
p3 <- plotReducedDim(cdScFiltAnnot, dimred = "MNN-TSNE", colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.5, theme_size = 20) +
    scale_color_manual(values = c_sample_col) +
    guides(color = guide_legend(title = "Sample", override.aes = list(size = 4, alpha = 1))) +
    ggtitle("t-SNE coloured by sample")

p3 <- oldScaterPlotFix(p3)
p3

pdf("tSNE_Sample.pdf")
p3
dev.off()

In [ ]:
# Creat breaks
bk <- seq(min(cdScFiltAnnot$log10Sum), max(cdScFiltAnnot$log10Sum), max(cdScFiltAnnot$log10Sum)/20)
bk <- round(bk, 2)

## t-SNE with lib size
p4 <- plotReducedDim(cdScFiltAnnot, dimred = "MNN-TSNE", colour_by = "log10Sum", text_by = "graphClusters", 
                     point_size = 1, text_size = 10, text_colour = "black", point_alpha = 0.5, theme_size = 20) +
    scale_color_gradientn(colours = rev(rainbow(5)), breaks = bk) +
    guides(color = guide_colorbar(title = "Library size", barheight = 15, 
                                  override.aes = list(size = 4, alpha = 1))) +
    ggtitle("t-SNE coloured by library size")

p4 <- oldScaterPlotFix(p4)
p4

In [ ]:
# Combine plots
fig(width = 16, height = 12)
plot_grid(p1, p2, p3, p4, ncol = 2, align = "vh")
reset.fig()

pdf("CellInfo_tsne.pdf")
plot_grid(p1, p2, p3, p4, ncol = 2, align = "vh")
dev.off()

<div class="alert alert-info">
  <strong>About this dataset: </strong> (edits required)
  <ul>
      <li><strike>Cluster 2 is dominated by high depth.</strike></li>
    <li><strike>It could happen that these cells go into one cluster becaue they are of higher read depth, indicating technical error. Or this could potentially indicate a sub-population that are quiescent cells.</strike></li>
  </ul>
</div>

In [ ]:
## UMAP with cluster allocation
p1 <- plotReducedDim(cdScFiltAnnot, dimred = "MNN-UMAP-Large", colour_by = "graphClusters", text_by = "graphClusters", 
                     point_size = 1, text_size = 10, text_colour = "black", point_alpha = 0.5, theme_size = 20) +
    scale_color_manual(values = c_clust_col) +
    guides(color = guide_legend(title = "Cluster", override.aes = list(size = 4, alpha = 1))) +
    ggtitle("UMAP coloured by clusters (graph-based)")

p1 <- oldScaterPlotFix(p1)
p1

pdf("UMAP_with_cluster.pdf")
p1
dev.off()

In [ ]:
## UMAP with Cell-cycle colouring
p2 <- plotReducedDim(cdScFiltAnnot, dimred = "MNN-UMAP-Large", colour_by = "CellCycle", 
                     point_size = 1, point_alpha = 0.5, theme_size = 20) +
    scale_color_manual(values = c_phase_col) +
    guides(color = guide_legend(title = "CellCycle", override.aes = list(size = 4, alpha = 1))) +
    ggtitle("UMAP coloured by cell-cycle")

p2 <- oldScaterPlotFix(p2)
p2

In [ ]:
## UMAP with sample colouring
p3 <- plotReducedDim(cdScFiltAnnot, dimred = "MNN-UMAP-Large", colour_by = "Sample", 
                     point_size = 1, point_alpha = 0.5, theme_size = 20) +
    scale_color_manual(values = c_sample_col) +
    guides(color = guide_legend(title = "Sample", override.aes = list(size = 4, alpha = 1))) +
    ggtitle("UMAP coloured by sample")

p3 <- oldScaterPlotFix(p3)
p3

pdf("UMAP_sample.pdf")
p3
dev.off()

In [ ]:
## UMAP with read-counts colouring
p4 <- plotReducedDim(cdScFiltAnnot, dimred = "MNN-UMAP-Large", colour_by = "log10Sum", text_by = "graphClusters", 
                     point_size = 1, text_size = 10, text_colour = "black", point_alpha = 0.5, theme_size = 20) +
    scale_color_gradientn(colours = rev(rainbow(5)), breaks = bk) +
    guides(color = guide_colorbar(title = "Library size", barheight = 15, 
                                  override.aes = list(size = 4, alpha = 1))) +
    ggtitle("UMAP coloured by library size")

p4 <- oldScaterPlotFix(p4)
p4

In [ ]:
# Combine plots
fig(width = 16, height = 12)
plot_grid(p1, p2, p3, p4, ncol = 2, align = "vh")
reset.fig()

pdf("CellInfo_UMAP.pdf")
plot_grid(p1, p2, p3, p4, ncol = 2, align = "vh")
dev.off()

__Note on interpreting plots__ Ideally clustering will determined by biology and not read count. Likewise the t-SNE and UMAP should not be dominated by the read depth. However, this is a known problem [Hafemeister et. al. 2019](https://doi.org/10.1101/576827).

In [ ]:
# How many cells from each sample are in each cluster?
table_samples_by_clusters <- table(Sample = cdScFiltAnnot$Sample, Cluster = cdScFiltAnnot$graphClusters)
table_samples_by_clusters

write.table(table_samples_by_clusters, file = "ClusterPopulations.xls", sep = "\t", quote = FALSE, col.names = NA)

In [ ]:
# No. of cells 
ggplot(data.frame(table_samples_by_clusters), aes(Sample, Freq, fill = Cluster)) + 
    geom_bar(position = "stack", stat = "identity") + 
    scale_fill_manual(values = c_clust_col) +
    scale_y_continuous("Number of cells", labels = comma) +
    theme_classic(base_size = 20) + theme(axis.title.x = element_blank())

In [ ]:
# Percenatge cells
ggplot(data.frame(table_samples_by_clusters), aes(Sample, Freq, fill = Cluster)) +
    geom_bar(position = "fill", stat = "identity") +
    scale_fill_manual(values = c_clust_col) +
    scale_y_continuous("Percentage [%]", labels = percent_format()) +
    theme_classic(base_size = 20) + theme(axis.title.x = element_blank())

## Visualizing gene expressions in cells

### Single gene expression

In the following plots we visualise the expression of `XIST` and `HNF4A` in individual cells.

In [ ]:
geneName <- "XIST"

p <- plotReducedDim(cdScFiltAnnot, "MNN-TSNE", colour_by = geneName, other_fields = "Sample",
                    point_size = 1, point_alpha = 0.5, theme_size = 20) +
    scale_color_viridis(option = "plasma", direction = -1) + facet_wrap(~ Sample) +
    guides(color = guide_colourbar(geneName)) + ggtitle(paste("t-SNE coloured by", geneName))

fig(width = 12, height = 10)
oldScaterPlotFix(p)
reset.fig()

In [ ]:
geneName <- "HNF4A"

p <- plotReducedDim(cdScFiltAnnot, "MNN-TSNE", colour_by = geneName, other_fields = "Sample",
                    point_size = 1, point_alpha = 0.5, theme_size = 20) +
    scale_color_viridis(option = "plasma", direction = -1) + facet_wrap(~ Sample) +
    guides(color = guide_colourbar(geneName)) + ggtitle(paste("t-SNE coloured by", geneName))

fig(width = 12, height = 10)
oldScaterPlotFix(p)
reset.fig()

### Multiple gene expression

In the following plots we visualise the expression of top 20 HVGs in individual cells using `ggplot`.

In [ ]:
geneNames <- head(hvg_genes, 20)

fig(width = 16, height = 11)
as.data.frame(reducedDim(cdScFiltAnnot, "MNN-TSNE")) %>% 
    bind_cols(as_tibble(t(logcounts(cdScFiltAnnot[geneNames,])))) %>%
    gather(., key = "Symbol", value ="Expression", -c(V1, V2) ) %>% 
    mutate_at(vars(Symbol), factor) %>%
    mutate(Symbol = factor(Symbol, levels = geneNames)) %>% # order by HVG gene list
    ggplot(aes(x = V1, y = V2, color = Expression)) + 
    geom_point(size = 0.3, alpha = 0.6) + facet_wrap(~ Symbol, ncol = 5) + 
    scale_color_viridis(option = "plasma", direction = -1) +
    theme_classic(base_size = 20) + theme(axis.title = element_blank())
reset.fig()

In [ ]:
fig(width = 12, height = 7)
plotDots(cdScFiltAnnot, features = geneNames, group = "graphClusters") + 
    scale_size(range = c(0.1, 5)) + theme_classic(base_size = 20) +
    scale_y_discrete(limits = geneNames) + # reorder genes
    labs(x = "Cluster", y = paste0("Top ", length(geneNames)," HVGs"))
reset.fig()

Show expression profiles of manually selected genes.

In [ ]:
selected_genes = c('ACTA2', 'COL1A1', 'COL1A2', 'PDGFRB',  'NGFR', 'SPP1', 'CCBE1', 'ONECUT1', 
'FOXP1', 'STAB2', 'PTPRC', 'CSF1R', 'EPOR' ,'IKZF1')

# Check if genes are in the sce object
cbind(setNames(selected_genes %in% rownames(cdScFiltAnnot), selected_genes))

# Subset to available marker genes
selected_genes = selected_genes[selected_genes %in% rownames(cdScFiltAnnot)]

In [ ]:
# Plot expression in t-SNE
fig(width = 12, height = 10)
as.data.frame(reducedDim(cdScFiltAnnot, "MNN-TSNE")) %>% 
    bind_cols(as_tibble(t(logcounts(cdScFiltAnnot[selected_genes,])))) %>%
    gather(., key = "Symbol", value ="Expression", -c(V1, V2) ) %>% 
    ggplot(aes(x = V1, y = V2, color = Expression)) + 
    geom_point(size = 0.3, alpha = 0.6) + facet_wrap(~ Symbol) + 
    scale_color_viridis(option = "plasma", direction = -1) +
    theme_classic(base_size = 20) + theme(axis.title = element_blank())
reset.fig()

In [ ]:
fig(width = 12, height = 5)
plotDots(cdScFiltAnnot, features = selected_genes, group = "graphClusters") + 
    scale_size(range = c(0.1, 5)) + theme_classic(base_size = 20) +
    labs(x = "Cluster", y = "Genes")
reset.fig()

#### Add percentage of expressing cells of a given cluster to `rowData` for all clusters.

In [ ]:
for(clustID in 1:max(as.numeric(cdScFiltAnnot$graphClusters))) {
    # Set column name
    dfColName <- paste0("PercentClust", clustID)
    # Number of expressing cells of a given cluster
    n_cells_by_counts <- nexprs(cdScFiltAnnot[, cdScFiltAnnot$graphClusters == clustID], byrow = TRUE)
    # Total cells of a given cluster
    total_cells <- sum(cdScFiltAnnot$graphClusters == clustID)
    # Add percentage of expressing cells of a given cluster
    rowData(cdScFiltAnnot)[,dfColName] <- round(100*(n_cells_by_counts/total_cells), 2)
}

#### Add percentage of expressing cells of a given sample to `rowData` for all samples.

In [ ]:
for(sampleID in levels(factor(cdScFiltAnnot$Sample))) {
    # Set column name
    dfColName <- paste0("PercentSample", sampleID)
    # Number of expressing cells of a given sample
    n_cells_by_counts <- nexprs(cdScFiltAnnot[, cdScFiltAnnot$Sample == sampleID], byrow = TRUE)
    # Total cells of a given sample
    total_cells <- sum(cdScFiltAnnot$Sample == sampleID)
    # Add percentage of expressing cells of a given cluster
    rowData(cdScFiltAnnot)[,dfColName] <- round(100*(n_cells_by_counts/total_cells), 2)
}

In [ ]:
save.image("data.RData")

# 11. Marker gene detection<a name='section11' />

Potential marker genes are identified by taking the top set of DE genes from each pairwise comparison between clusters. The results are arranged into a single output table that allows a marker set to be easily defined for a user-specified size of the top set. For example, to construct a marker set from the top 10 genes of each comparison, one would filter `marker.set` to retain rows with Top less than or equal to 10.

<div class="alert alert-warning">
  <strong>Warning!</strong> Take note of any poor clusters in the silhouette plot and consider how trustworthy the marker genes actually are.
</div>

Here we are going to use `findMarkers` from `scran` package as this is much faster. Setting `pval.type = "all"` gives genes that are significantly different against all other groups. 

## Find marker genes for clusters

In [ ]:
# Set outfile ID
file_id = "RachelJennings_aggr"
file_id

In [ ]:
# Only use is_pass and non-ambient genes
marker.genes.cluster <- findMarkers(cdScFiltSub, groups = cdScFiltSub$graphClusters, pval.type = "all", 
                                    BPPARAM = bpp)
marker.genes.cluster

In [ ]:
for(clustID in names(marker.genes.cluster)) {
    # Append Ensembl ID and Symbol from rowData
    marker.genes.cluster[[clustID]] <- cbind(rowData(cdScFiltSub)[rownames(marker.genes.cluster[[clustID]]),][,1:2], 
                                             marker.genes.cluster[[clustID]])
    write.table(marker.genes.cluster[[clustID]], 
                file = paste0(file_id, "_Cluster", clustID, ".tsv"), 
                sep = "\t", quote = FALSE, col.names = NA)
}

In [ ]:
# Up-regulated
# Only use is_pass and non-ambient genes
marker.genes.cluster.up <- findMarkers(cdScFiltSub, groups = cdScFiltSub$graphClusters, pval.type = "all", 
                                       lfc = 0.5, direction = "up", BPPARAM = bpp)
marker.genes.cluster.up

In [ ]:
for(clustID in names(marker.genes.cluster.up)) {
    # Append Ensembl ID and Symbol from rowData
    marker.genes.cluster.up[[clustID]] <- cbind(rowData(cdScFiltSub)[rownames(marker.genes.cluster.up[[clustID]]),][,1:2], 
                                                marker.genes.cluster.up[[clustID]])
    write.table(marker.genes.cluster.up[[clustID]],
                file = paste0(file_id, "_Upregulated_LFC_0.5_Cluster", clustID, ".tsv"), 
                sep = "\t", quote = FALSE, col.names = NA)
}

In [ ]:
metadata(cdScFiltAnnot)[['Cluster_markers']] <- list(marker.genes.cluster)

In [ ]:
metadata(cdScFiltAnnot)[['Cluster_markers_up']] <- list(marker.genes.cluster.up)

## Find marker genes for Samples

<div class="alert alert-danger">
  <strong>Danger!</strong> Only works for 2 or more samples.
</div>

In [ ]:
# Only use is_pass and non-ambient genes
marker.genes.sample <- findMarkers(cdScFiltSub, groups = cdScFiltSub$Sample, pval.type = "all", BPPARAM = bpp)
marker.genes.sample

In [ ]:
for(sampleID in names(marker.genes.sample)) {
    # Append Ensembl ID and Symbol from rowData
    marker.genes.sample[[sampleID]] <- cbind(rowData(cdScFiltSub)[rownames(marker.genes.sample[[sampleID]]),][,1:2], 
                                             marker.genes.sample[[sampleID]])
    write.table(marker.genes.sample[[sampleID]], 
                file = paste0(file_id, "_Sample_", sampleID, ".tsv"), 
                sep = "\t", quote = FALSE, col.names = NA)
}

In [ ]:
# Up-regulated
# Only use is_pass and non-ambient genes
marker.genes.sample.up <- findMarkers(cdScFiltSub, groups = cdScFiltSub$Sample, pval.type = "all", 
                                      lfc = 0.5, direction = "up", BPPARAM = bpp)
marker.genes.sample.up

In [ ]:
for(sampleID in names(marker.genes.sample.up)){
    # Append Ensembl ID and Symbol from rowData
    marker.genes.sample.up[[sampleID]] <- cbind(rowData(cdScFiltSub)[rownames(marker.genes.sample.up[[sampleID]]),][,1:2], 
                                                marker.genes.sample.up[[sampleID]])
    write.table(marker.genes.sample.up[[sampleID]],
                file = paste0(file_id, "_Upregulated_LFC_0.5_Sample_", sampleID, ".tsv"), 
                sep = "\t", quote = FALSE, col.names = NA)
}

In [ ]:
metadata(cdScFiltAnnot)[['Sample_markers']] <- list(marker.genes.sample)

In [ ]:
metadata(cdScFiltAnnot)[['Sample_markers_up']] <- list(marker.genes.sample.up)

## Plotting top 3 upregulated Cluster marker genes

The `plotDots` function create a dot plot of expression values for a grouping of cells, where the size and colour of each dot represents the proportion of detected expression values and the average expression, respectively, for each feature in each group of cells.

In [ ]:
noOfGenes <- 3 # total genes will be noOfGenes x number of clusters
geneNames <- sapply(marker.genes.cluster.up, function(x) rownames(x[1:noOfGenes,]))

print(paste("Number of genes to plot:", length(geneNames)))
geneNames # A matrix
                    
geneNames <- as.character(geneNames)

In [ ]:
fig(width = 14, height = 12)
plotDots(cdScFiltAnnot, features = geneNames, group = "graphClusters") +
    scale_size(range = c(0.1, 5)) + theme_classic(base_size = 20) + 
    scale_y_discrete(limits = geneNames) + # reorder genes
    labs(x = "Cluster", y = "Genes")
reset.fig()

### Heatmap for upregulated Cluster marker genes.

In [ ]:
fig(width = 16, height = 10)
if(packageVersion("scater") == "1.16.2") {
    plotHeatmap(cdScFiltAnnot, features = geneNames, 
                order_columns_by = "graphClusters", clustering_method = "ward.D2", show_colnames = FALSE)
} else {
    plotHeatmap(cdScFiltAnnot, features = geneNames, 
                order_columns_by = "graphClusters", clustering_method = "ward.D2", 
                column_annotation_colors = graph_colors, show_colnames = FALSE)
}
reset.fig()

### Stacked violin plot for upregulated Cluster marker genes.

In [ ]:
geneExprs <- assay(cdScFiltAnnot, "logcounts")[geneNames,]
geneExprs <- as.data.frame(t(as.matrix(geneExprs)))

geneExprs$Cell <- rownames(geneExprs)
geneExprs$Cluster <- cdScFiltAnnot$graphClusters

geneExprs <- reshape2::melt(geneExprs, id.vars = c("Cell","Cluster"), measure.vars = geneNames, 
                            variable.name = "Feature", value.name = "Expression") %>% 
    mutate(Cluster = fct_rev(Cluster)) # reverse order
head(geneExprs)

In [ ]:
# Plot
fig(width = 16, height = 10)
ggplot(geneExprs, aes(factor(Feature), Expression, fill = Feature)) +
        geom_violin(scale = "width", adjust = 1, trim = TRUE) +
        scale_y_continuous(expand = c(0, 0), position = "right", labels = function(x)
                           c(rep(x = "", times = length(x)-2), x[length(x) - 1], "")) +
        facet_grid(rows = vars(Cluster), scales = "free", switch = "y") +
        theme_cowplot(font_size = 16) +
        theme(legend.position = "none", panel.spacing = unit(0, "lines"),
              panel.background = element_rect(fill = NA, color = "black"),
              strip.background = element_blank(),
              strip.text = element_text(face = "bold"),
              strip.text.y.left = element_text(angle = 0),
              axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5)) +
        xlab("Feature") + ylab("Expression Level")
reset.fig()

## Visualizing top upregulated Cluster marker genes in t-SNE

In [ ]:
geneNames <- sapply(marker.genes.cluster.up, function(x) rownames(x[1,]))
                    
print(paste("Number of genes to plot:", length(geneNames)))
geneNames

In [ ]:
fig(width = 14, height = 12)
as.data.frame(reducedDim(cdScFiltAnnot, "MNN-TSNE")) %>% 
    bind_cols(as_tibble(t(logcounts(cdScFiltAnnot[geneNames,])))) %>%
    gather(., key = "Symbol", value = "Expression", -c(V1, V2) ) %>% 
    mutate_at(vars(Symbol), factor) %>%
    mutate(Symbol = factor(Symbol, levels = geneNames)) %>% # order by cluster id
    ggplot(aes(x = V1, y = V2, color = Expression)) + 
    geom_point(size = 0.3, alpha = 0.3) + 
    facet_wrap(~ Symbol, labeller = as_labeller(function(x) paste0(names(geneNames),": ", x))) + # add cluster id
    scale_color_viridis(option = "plasma", direction = -1) +
    theme_classic(base_size = 20) + theme(axis.title = element_blank())
reset.fig()

In [ ]:
fig(width = 12, height = 5)
plotDots(cdScFiltAnnot, features = geneNames, group = "graphClusters") + 
    scale_size(range = c(0.1, 5)) + theme_classic(base_size = 20) + 
    scale_y_discrete(limits = geneNames) + # reorder genes
    labs(x = "Cluster", y = "Genes")
reset.fig()

In [ ]:
save.image("data.RData")

## Enriched pathways

We will use `enrichR` package to access enrichr website in order to find the enriched pathways associated with __upregulated marker genes__. The results will be showed in the app.

In [ ]:
# This function generates the whole list of database for the enrichR. 
dbs <- listEnrichrDbs()
print(paste("Number of available databases from Enrichr:", nrow(dbs)))
head(dbs)

I will select the following list of databases. User can change depending on the Biological question and organism. The whole list is stored in the `dbs` object.

In [ ]:
dbsSel <- c("GO_Molecular_Function_2018", 
            "GO_Cellular_Component_2018", 
            "GO_Biological_Process_2018", 
            "BioPlanet_2019", 
            "MSigDB_Hallmark_2020", 
            "KEGG_2019_Human", 
            "KEGG_2019_Mouse", 
            "Human_Gene_Atlas",
            "Mouse_Gene_Atlas")

### Run analysis on cluster-specific marker genes

In [ ]:
resEnrichClust <- list()
for(clustID in names(marker.genes.cluster.up)) {
    df <- marker.genes.cluster.up[[clustID]] # findMarkers result DataFrame
    geneNames <- df[df$FDR <= 0.01,]$Symbol
    
    # Run enrichr if there are more than 1 genes satisfying the criteria
    if(length(geneNames) > 1) {
        print(paste("Running enrichr on cluster", clustID, "with", length(geneNames), "genes"))
        resEnrich <- enrichr(geneNames, databases = dbsSel)
        resEnrichTemp <- lapply(resEnrich, FDRsubsetting <- function(x) { 
            x <- x[x$Adjusted.P.value < 0.1, c(1:4)] 
            return(x)
        })
        resEnrichClust[[clustID]] <- resEnrichTemp
    }
}

metadata(cdScFiltAnnot)[['enrichCluster']] <- list(resEnrichClust)

### Run analysis on sample-specific marker genes

<div class="alert alert-danger">
  <strong>Danger!</strong> Only works for 2 or more samples.
</div>

In [ ]:
resEnrichSample <- list()
for(sampleID in names(marker.genes.sample.up)){
    df <- marker.genes.sample.up[[sampleID]] # findMarkers result DataFrame
    geneNames <- df[df$FDR <= 0.01,]$Symbol
    
    # Run enrichr if there are more than 1 genes satisfying the criteria
    if(length(geneNames) > 1) {
        print(paste("Running enrichr on sample", sampleID, "with", length(geneNames), "genes"))
        resEnrich <- enrichr(geneNames, databases = dbsSel)
        resEnrichTemp <- lapply(resEnrich, FDRsubsetting <- function(x){ 
            x <- x[x$Adjusted.P.value < 0.1, c(1:4)]
            return(x)
        })
        resEnrichSample[[sampleID]] <- resEnrichTemp
    }
}

metadata(cdScFiltAnnot)[['enrichSample']] <- list(resEnrichSample)

In [ ]:
save.image("data.RData")

## IPA

### Each cluster against other cells pooled for IPA

Here we generate the file that can be directly imported into IPA for donwstream analysis.

In [ ]:
resIPA <- marker.genes.cluster.up[[1]][,1:2]
resIPA <- resIPA[order(rownames(resIPA)),]

for(clustID in names(marker.genes.cluster.up)) {
    resTmp <- marker.genes.cluster.up[[clustID]][,3:5]
    resTmp <- resTmp[order(rownames(resTmp)),]
    colnames(resTmp) <- paste0("Cl_",clustID,"_",colnames(resTmp))
    resIPA <- cbind(resIPA, resTmp)
}

head(resIPA[,1:8])
write.table(resIPA, file = "IPA_each_cluster_v_other_cells.xls", sep = "\t", quote = FALSE, col.names = NA)

### Average expression (log2 CPM) across clusters

Here we are generating the mean value for genes (limits to `is_pass` genes) across clusters.

In [ ]:
system.time({
    table_count_by_cluster <- as_tibble(t(logcounts(cdScFiltAnnot[rowData(cdScFiltAnnot)$is_pass,]))) %>% 
    add_column(Cluster = cdScFiltAnnot[rowData(cdScFiltAnnot)$is_pass,]$graphClusters) %>% 
    group_by(Cluster) %>% summarise_all(mean, na.rm = TRUE) %>% t()
})

# nrows = genes; ncols = number of clusters
dim(table_count_by_cluster)

In [ ]:
write.table(table_count_by_cluster, file = "Cluster_mean_logcounts.xls", 
            sep = "\t", quote = FALSE, col.names = NA)

### Average expression (log2 CPM)  across Samples

Here we are generating the mean value for genes (limits to `is_pass` genes) across samples.

In [ ]:
system.time({
    table_count_by_sample <- as_tibble(t(logcounts(cdScFiltAnnot[rowData(cdScFiltAnnot)$is_pass,]))) %>%
    mutate(Sample_clust = paste0(as.character(cdScFiltAnnot[rowData(cdScFiltAnnot)$is_pass,]$Sample), "_", 
                                 as.character(cdScFiltAnnot[rowData(cdScFiltAnnot)$is_pass,]$graphClusters))) %>%
    group_by(Sample_clust) %>% summarise_all(mean, na.rm = TRUE) %>% t()
})

# nrows = genes; ncols = number of samples x number of clusters
dim(table_count_by_sample)

In [ ]:
write.table(table_count_by_sample, file = "Sample_Cluster_mean_logcounts.xls", 
            sep = "\t", quote = FALSE, col.names = NA)

In [ ]:
save.image("data.RData")

# 12. Cell type annotation<a name='section12' />

In this section, we will use of the `SingleR` method for cell type annotation. This method assigns labels to cells based on the reference samples with the highest Spearman rank correlations, using only the marker genes between pairs of labels to focus on the relevant differences between cell types. 

The `celldex` package contains a number of curated reference datasets (listed below), mostly assembled from bulk RNA-seq or microarray data of sorted cell types. 

<div class="alert alert-info">
    <strong>Info!</strong> In newer version of <code>SingleR</code>, the datasets are made available in the <code>celldex</code> package.
</div>


### Human

Following human reference datasets are available: 

- **BlueprintEncodeData** - Blueprint Epigenomics contains 144 RNA-seq pure immune samples annotated to 28 cell types. ENCODE contains 115 RNA-seq pure stroma and immune samples annotated to 17 cell types. All together, this reference contains 259 samples with 43 cell types ("`label.fine`"), manually aggregated into 24 broad classes ("`label.main`"). 

- **DatabaseImmuneCellExpressionData** - The dataset contains 1561 human RNA-seq samples annotated to 5 main cell types ("`label.main`"). Samples were additionally annotated to 15 fine cell types ("`label.fine`").

- **HumanPrimaryCellAtlasData** - The dataset contains 713 microarray samples from the Human Primary Cell Atlas (HPCA) (Mabbott et al., 2013). Each sample has been assigned to one of 37 main cell types ("`label.main`") and 157 subtypes ("`label.fine`"). 

- **MonacoImmuneData** - The dataset contains 114 human RNA-seq samples annotated to 10 main cell types ("`label.main`"). Samples were additionally annotated to 29 fine cell types ("`label.fine`").

- **NovershternHematopoieticData** - The dataset contains 211 human microarray samples annotated to 16 main cell types ("`label.main`"). Samples were additionally annotated to 38 fine cell types ("`label.fine`").

For specific applications, smaller datasets can be applicable. *SingleR* is flexible to be used with any reference dataset.

### Mouse:

Following mouse reference datasets are available: 

- **ImmGenData** - The dataset contains 830 microarray samples generated by ImmGen from pure populations of murine immune cells (<http://www.immgen.org/>). This dataset consists of 20 broad cell types ("`label.main`") and 253 finely resolved cell subtypes ("`label.fine`").

- **MouseRNAseqData** - The dataset contains 358 mouse RNA-seq samples annotated to 18 main cell types ("`label.main`"). These are split further into 28 subtypes ("`label.fine"`).

<div class="alert alert-warning">
  <strong>Warning!</strong> Edits required to choose human or mouse datasets.
</div>

In [ ]:
# Use celldex if installed, else falls back to SingleR
if ( requireNamespace("celldex", quietly = TRUE) ) {
    # For Human
    bpen <- celldex::BlueprintEncodeData()
#   dice <- celldex::DatabaseImmuneCellExpressionData()
    hpca <- celldex::HumanPrimaryCellAtlasData()
#   mona <- celldex::MonacoImmuneData()
#   dmap <- celldex::NovershternHematopoieticData()

    # For mouse
#   immg <- celldex::ImmGenData()
#   mmrna <- celldex::MouseRNAseqData()
} else {
    # For Human
    bpen <- SingleR::BlueprintEncodeData()
#   dice <- SingleR::DatabaseImmuneCellExpressionData()
    hpca <- SingleR::HumanPrimaryCellAtlasData()
#   mona <- SingleR::MonacoImmuneData()
#   dmap <- SingleR::NovershternHematopoieticData()

    # For mouse
#   immg <- SingleR::ImmGenData()
#   mmrna <- SingleR::MouseRNAseqData()    
}

We will use the Wilcoxon ranked sum test (`de.method`) to identify the top markers for each pairwise comparison between labels.

<div class="alert alert-warning">
  <strong>Warning!</strong> Please adapt the following codes and change the reference datasets to suit your dataset.
</div>

In [ ]:
# For human
pred.singler.bp <- SingleR(
    test = cdScFiltSub, # only use is_pass and non-ambient genes
#    clusters = colLabels(cdScFiltSub), # To obtain per-cluster profiles
    assay.type.test = "logcounts",
    de.method = "wilcox",
    ref = list(BP = bpen), 
    labels = list(bpen$label.main)
)

pred.singler.hpca <- SingleR(
    test = cdScFiltSub,
#    clusters = colLabels(cdScFiltSub),
    assay.type.test = "logcounts",
    de.method = "wilcox",
    ref = list(HPCA = hpca), 
    labels = list(hpca$label.main)
)

pred.singler <- SingleR(
    test = cdScFiltSub,
#    clusters = colLabels(cdScFiltSub),
    assay.type.test = "logcounts",
    de.method = "wilcox",
    ref = list(BP = bpen, HPCA = hpca), 
    labels = list(bpen$label.main, hpca$label.main)
)

# For mouse
#pred.singler <- SingleR(
#    test = cdScFiltSub,
#    clusters = colLabels(cdScFiltSub),
#    assay.type.test = "logcounts",
#    de.method = "wilcox",
#    ref = list(MouseRNA = mmrna), 
#    labels = list(mmrna$label.fine),
#)

## Inspect predicted labels

### Print cell type frequency

In [ ]:
table(BP = pred.singler.bp$labels)
table(HPCA = pred.singler.hpca$labels)
table(Combined = pred.singler$labels)

### Rename combined prediction labels

In [ ]:
# Re-label for consistency
HPCA_to_BP_labels <- function(pred = pred.singler) {
    HPCA <- c("Astrocyte","B_cell","T_cells","Endothelial_cells","Epithelial_cells",
              "Macrophage","Monocyte","NK_cell","Smooth_muscle_cells")
    BP <- c("Astrocytes","B-cells","T-cells","Endothelial cells","Epithelial cells",
            "Macrophages","Monocytes","NK cells","Smooth muscle")
    
    for(i in 1:length(HPCA)) {
        if(sum(pred$labels == HPCA[i]) > 0) {
            pred[pred$labels == HPCA[i],]$labels = BP[i]
        }
    }
    
    return(pred)
}

pred.singler <- HPCA_to_BP_labels(pred.singler)

### Show prediction scores

We use `plotScoreHeatmap()` to display the scores for all cells across all reference labels, which allows us to inspect the confidence of the predicted labels across the dataset. 

Ideally, each cell (i.e., column of the heatmap) should have one score that is obviously larger than the rest, indicating that it is unambiguously assigned to a single label. A spread of similar scores for a given cell indicates that the assignment is uncertain, though this may be acceptable if the uncertainty is distributed across similar cell types that cannot be easily resolved.

In [ ]:
fig(width = 16, height = 6)
plotScoreHeatmap(pred.singler.bp, grid.vars = list(ncol = 2))
reset.fig()

In [ ]:
fig(width = 16, height = 9)
plotScoreHeatmap(pred.singler.hpca, grid.vars = list(ncol = 2))
reset.fig()

In [ ]:
fig(width = 16, height = 11)
# Save to obj to retrieve cell type colours later
p <- plotScoreHeatmap(pred.singler, scores.use = 0, # Show combine scores
                     fontsize = 11, fontsize_row = 12)
reset.fig()

In [ ]:
# Assign cell type
#colData(cdScFiltAnnot)$CellType <- factor(pred.singler.bp)
#colData(cdScFiltAnnot)$CellType <- factor(pred.singler.hpca)
colData(cdScFiltAnnot)$CellType <- factor(pred.singler$labels) # Use BP + HPCA combined prediction
table(colData(cdScFiltAnnot)$CellType)

In [ ]:
# Use the same cell type colour from plotScoreHeatmap and store in c_celltype_col
celltype.df <- data.frame(Cell = colData(cdScFiltAnnot)$Barcode, 
                          CellType = colData(cdScFiltAnnot)$CellType) %>% arrange(Cell)

celltype.df$color <- data.frame(Cell = rownames(p$gtable$grobs[[5]]$gp[[1]]), 
                                color = p$gtable$grobs[[5]]$gp[[1]][,1]) %>% arrange(Cell) %>% pull(color) 

c_celltype_col <- distinct(celltype.df[,c("CellType","color")], CellType, .keep_all = TRUE)
c_celltype_col <- setNames(c_celltype_col$color, c_celltype_col$CellType)
c_celltype_col

In [ ]:
## t-SNE with Cell type
p <- plotReducedDim(cdScFiltAnnot, "MNN-TSNE", colour_by = "CellType", text_by = "graphClusters",
                    point_size = 1, text_size = 6, text_colour = "black", point_alpha = 0.5, theme_size = 20) +
    scale_color_manual(values = c_celltype_col) +
    guides(color = guide_legend(title = "CellType", ncol = 2, override.aes = list(size = 4, alpha = 1))) +
    ggtitle("t-SNE coloured by cell type")

fig(width = 16, height = 9)
oldScaterPlotFix(p)
reset.fig()

## Parallel sets visualisation

In [ ]:
# Use previously assigned colours
color_assignments <- c(c_sample_col, c_clust_col, c_phase_col, c_celltype_col)
color_assignments

#### Cell Cycle vs. Cell Type

In [ ]:
data <- as_tibble(colData(cdScFiltAnnot)[,c("CellCycle","CellType")]) %>% 
    group_by(CellCycle, CellType) %>% tally() %>% ungroup() %>% gather_set_data(1:2) %>% 
    mutate(x = factor(x, levels = c("CellCycle","CellType"))) # Cluster on the left and Cell type on the right

data_labels <- tibble(group = c(rep("CellCycle", length(levels(cdScFiltAnnot$CellCycle))), 
                                rep("CellType", length(levels(cdScFiltAnnot$CellType))))) %>%
    mutate(hjust = ifelse(group == "CellType", 0, 1), # CellType: left-justified, otherwise right-justified
           nudge_x = ifelse(group == "CellType", 0.1, -0.1)) # Horizontal nudge text labels

In [ ]:
p1 <- ggplot(data, aes(x, id = id, split = y, value = n)) + 
    geom_parallel_sets(aes(fill = CellCycle), alpha = 0.6, axis.width = 0.15) + 
    geom_parallel_sets_axes(aes(fill = y), color = "black", size = 0.3, axis.width = 0.1) + 
    geom_text(aes(y = n, split = y), stat = "parallel_sets_axes", hjust = data_labels$hjust, 
              nudge_x = data_labels$nudge_x, fontface = "bold", size = 5) + 
    scale_x_discrete(labels = c("Cell cycle phase", "Cell type")) + # Change x-axis labels
    scale_fill_manual(values = color_assignments) +
    theme_void(base_size = 20) + 
    theme(legend.position = "none", plot.margin = unit(c(1, 1, 1, 1), "lines"),
          axis.text.x = element_text(face = "bold", color = "black"))

fig(width = 14, height = 16)
p1
reset.fig()

In [ ]:
pdf("Parallel_sets_diagram_CellType_CellCycle.pdf")
p1
dev.off()

#### Hierarchical Cluster vs. Cell Type

In [ ]:
data <- as_tibble(colData(cdScFiltAnnot)[,c("hclustClusters","CellType")]) %>% 
    group_by(hclustClusters, CellType) %>% tally() %>% ungroup() %>% gather_set_data(1:2) %>% 
    mutate(x = factor(x, levels = c("hclustClusters","CellType"))) # Cluster on the left and Cell type on the right

data_labels <- tibble(group = c(rep("hclustClusters", length(levels(cdScFiltAnnot$hclustClusters))), 
                                rep("CellType", length(levels(cdScFiltAnnot$CellType))))) %>%
    mutate(hjust = ifelse(group == "hclustClusters", 1, 0), # label: right-justified, CellType: left-justified
           nudge_x = ifelse(group == "hclustClusters", -0.1, 0.1)) # Horizontal nudge labels

In [ ]:
p2 <- ggplot(data, aes(x, id = id, split = y, value = n)) + 
    geom_parallel_sets(aes(fill = hclustClusters), alpha = 0.6, axis.width = 0.15) + 
    geom_parallel_sets_axes(aes(fill = y), color = "black", size = 0.3, axis.width = 0.1) + 
    geom_text(aes(y = n, split = y), stat = "parallel_sets_axes", hjust = data_labels$hjust, 
              nudge_x = data_labels$nudge_x, fontface = "bold", size = 5) + 
    scale_x_discrete(labels = c("Cluster", "Cell type")) + # Change x-axis labels
    scale_fill_manual(values = color_assignments) +
    theme_void(base_size = 20) + 
    theme(legend.position = "none", plot.margin = unit(c(1, 1, 1, 1), "lines"),
          axis.text.x = element_text(face = "bold", color = "black"))

fig(width = 14, height = 16)
p2
reset.fig()

In [ ]:
pdf("Parallel_sets_diagram_CellType_hclustCluster.pdf")
p2
dev.off()

#### Graph-based Cluster vs. Cell Type

In [ ]:
data <- as_tibble(colData(cdScFiltAnnot)[,c("graphClusters","CellType")]) %>% 
    group_by(graphClusters, CellType) %>% tally() %>% ungroup() %>% gather_set_data(1:2) %>% 
    mutate(x = factor(x, levels = c("graphClusters","CellType"))) # Cluster on the left and Cell type on the right

data_labels <- tibble(group = c(rep("graphClusters", length(levels(cdScFiltAnnot$graphClusters))), 
                                rep("CellType", length(levels(cdScFiltAnnot$CellType))))) %>%
    mutate(hjust = ifelse(group == "graphClusters", 1, 0), # label: right-justified, CellType: left-justified
           nudge_x = ifelse(group == "graphClusters", -0.1, 0.1)) # Horizontal nudge labels

In [ ]:
p3 <- ggplot(data, aes(x, id = id, split = y, value = n)) + 
    geom_parallel_sets(aes(fill = graphClusters), alpha = 0.6, axis.width = 0.15) + 
    geom_parallel_sets_axes(aes(fill = y), color = "black", size = 0.3, axis.width = 0.1) + 
    geom_text(aes(y = n, split = y), stat = "parallel_sets_axes", hjust = data_labels$hjust, 
              nudge_x = data_labels$nudge_x, fontface = "bold", size = 5) + 
    scale_x_discrete(labels = c("Cluster", "Cell type")) + # Change x-axis labels
    scale_fill_manual(values = color_assignments) +
    theme_void(base_size = 20) + 
    theme(legend.position = "none", plot.margin = unit(c(1, 1, 1, 1), "lines"),
          axis.text.x = element_text(face = "bold", color = "black"))

fig(width = 14, height = 16)
p3
reset.fig()

In [ ]:
pdf("Parallel_sets_diagram_CellType_graphCluster.pdf")
p3
dev.off()

#### Sample vs. Cell Type

In [ ]:
data <- as_tibble(colData(cdScFiltAnnot)[,c("Sample","CellType")]) %>% 
    group_by(Sample, CellType) %>% tally() %>% ungroup() %>% gather_set_data(1:2) %>% 
    mutate(x = factor(x, levels = c("Sample","CellType"))) # Cluster on the left and Cell type on the right

data_labels <- tibble(group = c(rep("Sample", length(levels(cdScFiltAnnot$Sample))), 
                                rep("CellType", length(levels(cdScFiltAnnot$CellType))))) %>%
    mutate(hjust = ifelse(group == "CellType", 0, 1), # CellType: left-justified, otherwise right-justified
           nudge_x = ifelse(group == "CellType", 0.1, -0.1)) # Horizontal nudge text labels

In [ ]:
p4 <- ggplot(data, aes(x, id = id, split = y, value = n)) + 
    geom_parallel_sets(aes(fill = Sample), alpha = 0.6, axis.width = 0.15) + 
    geom_parallel_sets_axes(aes(fill = y), color = "black", size = 0.3, axis.width = 0.1) + 
    geom_text(aes(y = n, split = y), stat = "parallel_sets_axes", hjust = data_labels$hjust, 
              nudge_x = data_labels$nudge_x, fontface = "bold", size = 5) + 
    scale_x_discrete(labels = c("Sample", "Cell type")) + # Change x-axis labels
    scale_fill_manual(values = color_assignments) +
    theme_void(base_size = 20) + 
    theme(legend.position = "none", plot.margin = unit(c(1, 1, 1, 1), "lines"),
          axis.text.x = element_text(face = "bold", color = "black"))

fig(width = 14, height = 16)
p4
reset.fig()

In [ ]:
pdf("Parallel_sets_diagram_CellType_Sample.pdf")
p4
dev.off()

In [ ]:
saveHDF5SummarizedExperiment(cdScFiltAnnot, dir = "cdScFiltAnnotHDF5", prefix = "", replace = TRUE,
                             chunkdim = NULL, level = NULL, verbose = FALSE)

In [ ]:
save.image("data.RData")

# Session Info

In [ ]:
writeLines(capture.output(sessionInfo()), "sessionInfo.txt")
sessionInfo()

# References

1. Lun ATL, McCarthy DJ and Marioni JC. A step-by-step workflow for low-level analysis of single-cell RNA-seq data with Bioconductor. F1000Research (2016) 5:2122 (https://doi.org/10.12688/f1000research.9501.2)
2. Luecke MD and Theis FJ, Current best practices in single‐cell RNA‐seq analysis: a tutorial, Mol Syst Biol (2019) 15:e8746 https://doi.org/10.15252/msb.20188746)
3. Amezquita R, Lun ATL, Hicks S and Gottardo R. Orchestrating Single-Cell Analysis with Bioconductor. Version: 1.0.6; Compiled: 2020-12-08. (http://bioconductor.org/books/release/OSCA/)
4. Lun ATL. Aaron's single-cell thoughts (https://ltla.github.io/SingleCellThoughts/)
5. University of Cambridge Bioinformatics Training Unit. Analysis of single cell RNA-seq data. Compiled: 2019-07-01. (https://scrnaseq-course.cog.sanger.ac.uk/website/index.html)
6. Harvard Chan Bioinformatics Core. Introduction to Single-cell RNA-seq. (https://hbctraining.github.io/scRNA-seq/schedule/)